In [31]:
#!/usr/bin/env python
# coding: utf-8



import pynumad
import sys
# add Cubit libraries to your pathhas
sys.path.append('/home/ecamare/designCodes/Cubit-16.10/bin')
sys.path.append('/home/ecamare/designCodes/Cubit-16.10/bin/python3/lib/python3.7/site-packages/')
import cubit
#from cubitUtils import *
from PyCubed_Main import * 
from cubit import *
import math
import numpy as np
import re



#Assumptions to resolve later
#Exactly two webs are assumed. The forweb and aftweb is not robust. It is hardcoded.
#The presence of flatback stacks that will be ignored
#The spar caps stack has component with the name 'spar' (case insensitive)
#Assumes that the HP side is made first
#Only works with SI units

model2Dor3D='2D' #eithere 2D 3D. 2D makes cross-sections. 3D makes brick models
curve_flage=True #only used for model2Dor3D='2D'
myCase='B'
VABSfileNameBase='linuxcase-' + myCase
adhesiveComponentName='Adhesive'; #The name of the blade material for the adhesive.
crosssectionParams = {}
crosssectionParams['nelPerLayer'] = 3
crosssectionParams['elementAR'] = 5
crosssectionParams['elementShape'] = 'quad'

crosssectionParams['elementShapeFunction'] = 'linear'

crosssectionParams['transitionTaperAngle'] = 30
crosssectionParams['amplitudeFraction']=1.0


blade=pynumad.Blade()
blade.read_yaml('myBlade-bendTwist.yaml')
blade.expandBladeGeometryTEs()
blade.editStacksForSolidMesh()

#Search for the material ID of the adhesive. This will become the block ID
for iMat,material in enumerate(blade.materials):
    if material.name in adhesiveComponentName:
        crosssectionParams['adhesiveMatID'] = iMat



cubit.init(['cubit','-nojournal'])
cubit.cmd('reset ')
cubit.cmd('undo off')
cubit.cmd('set geometry accuracy 1e-6')
cubit.cmd('set default autosize off') #making numerus 3D volumes is very slow with autosize on


TEangleLimit=120
thicknessScaling=0.001
geometryScaling=thicknessScaling*1000
#maxCrossSectionSurfaceCT=0

surfaceDict={}

materialIDsUsed=set() #Uniquly track which materiall IDs are actuall used in blade model


iStationStart = 4
iStationEnd = 4
dStation = 1



def addColor(blade,volumeOrSurface):
    
    #Adds color to volume or surfaces by material
    colorDict={}
    colorDict['adhesive']='yellow'
    colorDict['uni']='grey'  
    colorDict['triax']='lightgreen' 
    colorDict['biax']='lightblue' 
    colorDict['foam']='orange'
    for imat,material in enumerate(blade.materials):
        matName=material.name.lower()
        for color in colorDict.keys():
            if color in matName:
                parseString=f'with name "*material{imat}*"'

                volIDs=parse_cubit_list(volumeOrSurface, parseString)
                cubit.cmd(f'color {volumeOrSurface} {l2s(volIDs)}  mesh {colorDict[color]}')
def surfaceFromTwoCurves(topCurve,bottomCurve):
    v2Left,v2Right=selCurveVerts(topCurve)
    v1Left,v1Right=selCurveVerts(bottomCurve)
    cubit.cmd(f'create curve vertex {v1Left} {v2Left}')
    cubit.cmd(f'create curve vertex {v1Right} {v2Right}')
    cubit.cmd(f'create surface curve {l2s([get_last_id("curve")-1, bottomCurve,topCurve,get_last_id("curve")])}')


def getCrossSectionNormalVector(xyz):
    npts,_=xyz.shape


    #Create Referece line as a spline
    vertexList=[]
    for kcp in range(npts):
        vertexList.append(create_vertex(xyz[kcp,0],xyz[kcp,1],xyz[kcp,2]))

    c1=create_curve(vertexList[0],vertexList[1])
    c2=create_curve(vertexList[0],vertexList[2])

    midPoint=list(c1.position_from_fraction(0.5))
    tangent1=c1.tangent(midPoint)
    midPoint=list(c2.position_from_fraction(0.5))
    tangent2=c2.tangent(midPoint)
    crossSectionNormal=vectNorm(crossProd(list(tangent1),list(tangent2)))
    cubit.cmd(f'delete curve {get_last_id("curve")}')
    cubit.cmd(f'delete curve {get_last_id("curve")-1}')
    return crossSectionNormal
def getBladeGeometryForStation(blade,iStation):
    return np.array([blade.geometry[:,0,iStation],blade.geometry[:,1,iStation],blade.geometry[:,2,iStation]]).transpose()


def writeSplineFromCoordinatePoints(xyz):
    
    #xyz is npts by 3 array holding the coordinates of the points
    npts,_=xyz.shape
    nStart=get_last_id("vertex")+1
    for iPoint in range(npts):
        coords=xyz[iPoint,:]
        create_vertex(coords[0],coords[1],coords[2])
    nEnd=get_last_id("vertex")
    vertexList=list(range(nStart,nEnd+1))
    cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')

def extendCurveAtVertexToLength(curveToExtendID,extensionLength,curveStartOrEnd): 
    #Extend all offset curves
    if curveStartOrEnd.lower() == 'start':
        tempVertID,_=selCurveVerts(curveToExtendID)
        vectorDirectionSign=-1
    else:
        _,tempVertID=selCurveVerts(curveToExtendID)
        vectorDirectionSign=1 
        
    tangentLocationCoords=cubit.vertex(tempVertID).coordinates()
        
    x=cubit.curve(curveToExtendID).tangent(tangentLocationCoords)[0]
    y=cubit.curve(curveToExtendID).tangent(tangentLocationCoords)[1]
    z=cubit.curve(curveToExtendID).tangent(tangentLocationCoords)[2]
    tangentDirection=vectorDirectionSign*extensionLength*np.array(vectNorm([x,y,z]))  #Unit vector of tangent *Scaled by offset curve length
    newVertexCoords=np.array(tangentLocationCoords)+tangentDirection
    
    v1=cubit.create_vertex(newVertexCoords[0],newVertexCoords[1],newVertexCoords[2])
    
    #if statement needed to maintain original curve sense
    if curveStartOrEnd.lower() == 'start':
        c1=cubit.create_curve(v1,cubit.vertex(tempVertID))
        #Combine offset with extenstion
        cubit.cmd(f'create curve combine curve {get_last_id("curve")} {curveToExtendID}')
    else:
        c1=cubit.create_curve(cubit.vertex(tempVertID),v1)
        #Combine offset with extenstion
        cubit.cmd(f'create curve combine curve {curveToExtendID} {get_last_id("curve")}')  
        
    cubit.cmd(f'delete curve {curveToExtendID} {get_last_id("curve")-1}')
    return get_last_id("curve")

def removeBadTEgeometry(curveID,flatBackCurveID):
    cubit.cmd(f'split curve {curveID} distance {blade.chord[iStation]*0.002} from start ')
    cubit.cmd(f'delete curve {get_last_id("curve")-1}')
    curveID=get_last_id("curve")

    curveStartOrEnd='start'
    extensionLength=1*cubit.curve(curveID).length()
    curveID=extendCurveAtVertexToLength(curveID,extensionLength,curveStartOrEnd)

    _,v1=selCurveVerts(curveID)
    cubit.cmd(f'trim curve {curveID} atintersection curve {flatBackCurveID} keepside vertex {v1}')
    return get_last_id("curve")

def printOffsetDirectionCheck(curveID,lpHpside,crossSectionNormal):
    #This function is used to determine which way a curve offset will go. This is needed, for example, 
    #to make sure the outer mold line curve is being offset towrads the interior of the blade. 
    tol=0.01
    cubit.cmd(f'create curve offset curve {curveID} distance 0.0001')
    tempCurveID=get_last_id("curve")
    cubit.cmd(f'create surface skin curve {curveID} {tempCurveID}')
    cubit.cmd(f'delete curve {tempCurveID}')
    n=get_surface_normal(get_last_id("surface"))
    if abs(n[0]-crossSectionNormal[0]) < tol and abs(n[1]-crossSectionNormal[1]) < tol and abs(n[2]-crossSectionNormal[2]) < tol:
        offsetSign=1
    else:
        offsetSign=-1
    cubit.cmd(f'delete body {get_last_id("body")}')
    
    #Need to flip direction since LP curve is written to run clockwise
    if lpHpside.lower() == 'lp':
        offsetSign=-1*offsetSign    
    return offsetSign

def offsetCurveAndCombineFragmentsIfNeeded(curveID,offsetDistance):
    #Sometimes when offseting a curve, the offset is broken into multiple curves. 
    #This happens when the curvature is to high for the offset distance. If this happens
    #this fuction combines the fragmented curves into one spline.
    nStart=get_last_id("curve")
    cubit.cmd(f'create curve offset curve {curveID} distance {offsetDistance} extended')
    nEnd=get_last_id("curve")
    if nEnd-nStart > 1:
        curveList=[]
        curveList+=list(range(nStart+1,nEnd+1))
        vertexList=[]
        v1,_=selCurveVerts(curveList[0])
        vertexList.append(v1)
        for curve in curveList:
            nStart=get_last_id("vertex")+1
            cubit.cmd(f'create vertex on curve {curve} segment 200')
            nEnd=get_last_id("vertex")
            vertexList+=list(range(nStart,nEnd+1))
        _,v1=selCurveVerts(curveList[-1])
        cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
        cubit.cmd(f'delete vertex {l2s(vertexList[1:-1])}')
        cubit.cmd(f'delete curve {l2s(curveList)}')
        


def streamlineCurveIntersections(firstCurveID,secondCurveID,keepCurve):
    #keepCurve is either 1 or 2
    nStart=get_last_id("vertex")
    cubit.cmd(f'create vertex atintersection curve {firstCurveID} {secondCurveID}')
    nEnd=get_last_id("vertex")
    vertexList=list(range(nStart+1,nEnd+1))
    if len(vertexList)>0:
        cubit.cmd(f'split curve {firstCurveID} at vertex {vertexList[-1]}')
        firstCurveID=get_last_id("curve")-1
        cubit.cmd(f'split curve {secondCurveID} at vertex {vertexList[-1]}')
        secondCurveID=get_last_id("curve")
        tempID=spliceTwoCurves(firstCurveID,secondCurveID,keepCurve)
        return tempID
    else:
        if keepCurve==1:
            return firstCurveID
        else:
            return secondCurveID
def spliceTwoCurves(firstCurveID,secondCurveID,keepCurve):
    #Given two curves splice them into one curve. This should even work for curves that make corners.
    #Curve sence matters. The first curve's sense (tangent) should point towards the second curve.
    cubit.cmd(f'split curve {firstCurveID} distance 0.005 from end ')
    firstCurveID=get_last_id("curve")-1
    cubit.cmd(f'split curve {secondCurveID} distance 0.005 from start ')
    secondCurveID=get_last_id("curve")
    vertexList=[]
    v1,_=selCurveVerts(firstCurveID)
    vertexList.append(v1)
    nStart=get_last_id("vertex")+1
    cubit.cmd(f'create vertex on curve {firstCurveID} segment 200')
    nEnd=get_last_id("vertex")
    vertexList+=list(range(nStart,nEnd+1))
    _,v1=selCurveVerts(firstCurveID)
    vertexList.append(v1)
    v2,_=selCurveVerts(secondCurveID)
    vertexList.append(v2)
    nStart=get_last_id("vertex")+1
    cubit.cmd(f'create vertex on curve {secondCurveID} segment 200')
    nEnd=get_last_id("vertex")
    vertexList+=list(range(nStart,nEnd+1))
    _,v2=selCurveVerts(secondCurveID)
    vertexList.append(v2)
    cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
    cubit.cmd(f'delete curve {firstCurveID} {secondCurveID}')
    secondCurveID=get_last_id("curve")
    cubit.cmd(f'delete vertex {l2s(vertexList[1:-1])}')
    if keepCurve==1:
        return firstCurveID
    elif keepCurve==2:
        return secondCurveID
def extendCurvePastCurveAndTrim(curveToExtendID,curveStartOrEnd,curveIDThatCutsExtendedCurve):
    #Given two curves that are not necessarily intersecting extend {curveToExtendID} then trim it at
    #{curveIDThatCutsExtendedCurve}. {curveStartOrEnd} defines which side of the curve to extend so 
    #you need to know the curve sense

    extensionLength=2*cubit.curve(curveToExtendID).length()
    curveToExtendID=extendCurveAtVertexToLength(curveToExtendID,extensionLength,curveStartOrEnd)  
    
    nStart=get_last_id("vertex")
    cubit.cmd(f'create vertex AtIntersection curve {curveIDThatCutsExtendedCurve}  {curveToExtendID}')
    splitVertexID=get_last_id("vertex")
    cubit.cmd(f'split curve {curveToExtendID} at vertex {splitVertexID}')
    
    if curveStartOrEnd.lower() == 'start':
        curveToExtendID=get_last_id("curve")
        cubit.cmd(f'delete curve {get_last_id("curve")-1}')
    else:
        curveToExtendID=get_last_id("curve")-1
        cubit.cmd(f'delete curve {get_last_id("curve")}')   
    return curveToExtendID
def renameLastSurface(partName, iStation,iModeledLayers,materialID,partNameID):
    #Every cross sectional surface that is created must be followed by a call to this function
    partNameID+=1
    surfaceName=partName+'Station'+str(iStation)+'_layer'+str(iModeledLayers)+'_material'+str(materialID)+'_surface'+str(partNameID)
    cubit.cmd(f'surface {get_last_id("surface")} rename "{surfaceName}"')
    return partNameID

def addSurfaceDictEntry(surfaceObject,myCurveOrder, myVertOrder,materialID,plyAngle):
    surfaceDict[surfaceObject.id()]={}

    #Curves:
    idList=[]
    for curveObject in surfaceObject.curves():
        idList.append(curveObject.id())
    idList = [idList[i] for i in myCurveOrder]

    surfaceDict[surfaceObject.id()]['curves']=idList
    # getSurfaceCurves[surfaceObject.id()]=idList
    
    #Verts:
    idList=[]
    for vertObject in surfaceObject.vertices():
        idList.append(vertObject.id())
    idList = [idList[i] for i in myVertOrder]
    surfaceDict[surfaceObject.id()]['verts']=idList
    surfaceDict[surfaceObject.id()]['materialID']=materialID
    surfaceDict[surfaceObject.id()]['plyAngle']=plyAngle
    # getSurfaceVerts[surfaceObject.id()]=idList
    
def makeCrossSectionSurface(partName,topCurve,bottomCurve,materialID,plyAngle,partNameID,iModeledLayers,materialIDsUsed):  
    #Given two curves in a cross section, create a surface by connecting the end points then
    #rename the surface and add to the surface dictionary
    surfaceFromTwoCurves(topCurve,bottomCurve)
    materialIDsUsed.add(materialID)
    partNameID=renameLastSurface(partName,iStation,iModeledLayers,materialID,partNameID)
    addSurfaceDictEntry(cubit.surface(get_last_id("surface")),[0,1,2,3],[0,1,2,3],materialID,plyAngle) 
    
    cubit.cmd(f'curve {surfaceDict[get_last_id("surface")]["curves"][1]} rename "layerThickness"')
    cubit.cmd(f'curve {surfaceDict[get_last_id("surface")]["verts"][-1]} rename "layerThickness"')

    
    return partNameID,materialIDsUsed


def writeLEAdhesiveCurves(HPstackThickness,LPstackThickness,adhesiveThickness,hpKeyCurve,lpKeyCurve,crossSectionNormal): 
    def uniteLEgeomWithAdhesiveCurves(lpHpside,keyCurve):
        if lpHpside.lower() == 'hp':
            #keyCurve=hpKeyCurve
            adhesiveCurveID=LEadhesiveCurveIDs[0][0]
            offsetCurve=hpOffset
        else:
            #keyCurve=lpKeyCurve
            adhesiveCurveID=LEadhesiveCurveIDs[1][0]
            offsetCurve=lpOffset
            
        print(f'lpHpside: {lpHpside} keyCurve {keyCurve}')

            
        v1,_=selCurveVerts(keyCurve)

        cubit.cmd(f'trim curve {keyCurve} atintersection curve {adhesiveCurveID} keepside vertex {v1}')
        keyCurve=get_last_id("curve")
        
        v1,_=selCurveVerts(offsetCurve)
        cubit.cmd(f'trim curve {offsetCurve} atintersection curve {adhesiveCurveID} keepside vertex {v1}')
        
        offsetCurve=get_last_id("curve")
        _,v1=selCurveVerts(keyCurve)
        _,v2=selCurveVerts(offsetCurve)
        
        cubit.cmd(f'create curve spline vertex {v1} {v2}')
        cubit.cmd(f'delete curve {adhesiveCurveID}')
        cubit.cmd(f'delete curve {offsetCurve}')
        
        return get_last_id("curve"), keyCurve
    
    LEadhesiveCurveIDs=[[],[]]
    
    
    _,p1=selCurveVerts(hpKeyCurve)
    _,p2=selCurveVerts(lpKeyCurve)
    
    coords=[]
    coords.append(list(cubit.vertex(p1).coordinates()))
    coords.append(list(cubit.vertex(p2).coordinates()))
    coords=np.array(coords)
    coords=np.mean(coords,0)
    v1=cubit.create_vertex(coords[0],coords[1],coords[2])
    
    cubit.cmd(f'vertex {get_last_id("vertex")} copy')
    midPointOML=get_last_id("vertex")
    
    #Offset OML curves to final layer offset
    cubit.cmd(f'curve {hpKeyCurve} copy')
    cubit.cmd(f'split curve {get_last_id("curve")} fraction 0.05 from end')
    cubit.cmd(f'create curve offset curve {get_last_id("curve")} distance {-1*HPstackThickness} extended')
    _,p1=selCurveVerts(get_last_id("curve"))
    hpOffset=get_last_id("curve")
    
    cubit.cmd(f'curve {lpKeyCurve} copy')
    cubit.cmd(f'split curve {get_last_id("curve")} fraction 0.05 from end')
    cubit.cmd(f'create curve offset curve {get_last_id("curve")} distance {-1*LPstackThickness} extended')
    _,p2=selCurveVerts(get_last_id("curve"))
    lpOffset=get_last_id("curve")
    
    coords=[]
    coords.append(list(cubit.vertex(p1).coordinates()))
    coords.append(list(cubit.vertex(p2).coordinates()))
    coords=np.array(coords)
    coords=np.mean(coords,0)
    v2=cubit.create_vertex(coords[0],coords[1],coords[2])
    c1=cubit.create_curve(v1,v2)
    adhesiveMidLine=get_last_id("curve")
    
    #Extend midline on both sides to make sure other curves eventually intersect with it 
    curveStartOrEnd='start'
    extensionLength=2*cubit.curve(adhesiveMidLine).length()
    adhesiveMidLine=extendCurveAtVertexToLength(adhesiveMidLine,extensionLength,curveStartOrEnd)
    
    curveStartOrEnd='end'
    extensionLength=1*cubit.curve(adhesiveMidLine).length()
    adhesiveMidLine=extendCurveAtVertexToLength(adhesiveMidLine,extensionLength,curveStartOrEnd)
    
    #Copy and move since offset does not seem to work with strait lines
    
    #get offset vector
    
    axialDirection=crossSectionNormal
    position = cubit.curve(adhesiveMidLine).position_from_fraction(1.0)
    tangentDirection=cubit.curve(adhesiveMidLine).tangent(position)
 
    normalDirection=crossProd(axialDirection,tangentDirection)
    normalDirection=adhesiveThickness/2*np.array(vectNorm([normalDirection[0],normalDirection[1],normalDirection[2]]))
    cubit.cmd(f'curve {adhesiveMidLine} copy move x {normalDirection[0]} y {normalDirection[1]} z {normalDirection[2]} nomesh')
    
    LEadhesiveCurveIDs[0].append(get_last_id("curve"))
    normalDirection=-1*normalDirection
    cubit.cmd(f'curve {adhesiveMidLine} copy move x {normalDirection[0]} y {normalDirection[1]} z {normalDirection[2]} nomesh')
    LEadhesiveCurveIDs[1].append(get_last_id("curve"))
    cubit.cmd(f'delete curve {adhesiveMidLine}')

    keyCurves=[hpKeyCurve,lpKeyCurve]
    for iSide, lpHpside in enumerate(['HP','LP']):
        
        ###HP###
        LEadhesiveCurveIDs[iSide][0],keyCurves[iSide]=uniteLEgeomWithAdhesiveCurves(lpHpside,keyCurves[iSide])

        #Make Copies
        cubit.cmd(f'curve {LEadhesiveCurveIDs[iSide][0]} copy')
        LEadhesiveCurveIDs[iSide].append(get_last_id("curve"))

        cubit.cmd(f'curve {LEadhesiveCurveIDs[iSide][1]} copy')
        LEadhesiveCurveIDs[iSide].append(get_last_id("curve"))
        #Extend
        curveStartOrEnd='end'


        extensionLength=1*cubit.curve(LEadhesiveCurveIDs[iSide][1]).length()
        LEadhesiveCurveIDs[iSide][1]=extendCurveAtVertexToLength(LEadhesiveCurveIDs[iSide][1],extensionLength,curveStartOrEnd)



    return keyCurves[0],keyCurves[1],LEadhesiveCurveIDs
def splitCurveAtCoordintePoints(coordinatesToSplitCurve,curveIDToSplit): 
    cubit.cmd(f'curve {curveIDToSplit} copy')
    tempCurveID=get_last_id("curve")
    

    nDPs,_=coordinatesToSplitCurve.shape
    idStart=get_last_id("vertex")+1
    for kcp in range(nDPs):
        temp=cubit.curve(tempCurveID).closest_point([coordinatesToSplitCurve[kcp,0],coordinatesToSplitCurve[kcp,1],coordinatesToSplitCurve[kcp,2]])
        create_vertex(temp[0],temp[1],temp[2])
    idEnd=get_last_id("vertex")
    
    DPverticies = [i for i in range(idStart,idEnd+1)]
    
    idStart=get_last_id("curve")+1
    cubit.cmd(f'split curve {tempCurveID} at vertex {l2s(DPverticies)}')
    idEnd=get_last_id("curve")
    keyCurves = [i for i in range(idStart,idEnd+1)]
    return keyCurves

def splitKeyCurves(keyCurves):
    ###Do not split TE reinf
    tempBaseCurveIDs=[keyCurves[0]]

    ###split TE panel in half
    cubit.cmd(f'split curve {keyCurves[1]} fraction 0.5')

    tempBaseCurveIDs.append(get_last_id("curve")-1)
    tempBaseCurveIDs.append(get_last_id("curve"))

    ###Partition sparcap curve
    vertexList=[]
    webLayerThickness=0
    nStart=get_last_id("vertex")+1
    for iLayer in reversed(range(len(aftWebStack.plygroups))):
        webLayerThickness+=aftWebStack.plygroups[iLayer].thickness*aftWebStack.plygroups[iLayer].nPlies/1000
        cubit.cmd(f'create vertex on curve {keyCurves[2]}  distance {webLayerThickness} from start')
    cubit.cmd(f'create vertex on curve {keyCurves[2]}  distance {webLayerThickness+crosssectionParams["Web_adhesive_width"]} from start')

    #get total foreweb thickness
    webLayerThickness=sum(foreWebStack.layerThicknesses())/1000
    cubit.cmd(f'create vertex on curve {keyCurves[2]}  distance {webLayerThickness+crosssectionParams["Web_adhesive_width"]} from end')
    for iLayer in reversed(range(len(foreWebStack.plygroups))):
        cubit.cmd(f'create vertex on curve {keyCurves[2]}  distance {webLayerThickness} from end')
        webLayerThickness-=foreWebStack.plygroups[iLayer].thickness*foreWebStack.plygroups[iLayer].nPlies/1000


    nEnd=get_last_id("vertex")
    vertexList+=list(range(nStart,nEnd+1))

    nStart=get_last_id("curve")+1
    cubit.cmd(f'split curve {keyCurves[2]} at vertex {l2s(vertexList)}')
    nEnd=get_last_id("curve")
    tempBaseCurveIDs.append(nStart)
    tempBaseCurveIDs.append(nEnd)
    sparCapBaseCurves=list(range(nStart+1,nEnd))

    ###split LE panel in half
    cubit.cmd(f'split curve {keyCurves[3]} fraction 0.5')
    tempBaseCurveIDs.append(get_last_id("curve")-1)
    tempBaseCurveIDs.append(get_last_id("curve"))

    ###Do not split LE reinf
    tempBaseCurveIDs.append(keyCurves[-1])

    return tempBaseCurveIDs,sparCapBaseCurves
def getMidLine(iLE):
    X=blade.geometry[:,0,iStation]* geometryScaling
    Y=blade.geometry[:,1,iStation]* geometryScaling
    Z=blade.geometry[:,2,iStation]* geometryScaling


    ###### Get averge line
    xHP = X[1:iLE]
    xLP = np.flip(X[iLE-1:-1])
    yHP = Y[1:iLE]
    yLP = np.flip(Y[iLE-1:-1])
    zHP = Z[1:iLE]
    zLP = np.flip(Z[iLE-1:-1])
    midline = np.zeros((len(xHP),3))
    for iPoint in range(len(xHP)):
        midline[iPoint,0] = (xHP[iPoint] + xLP[iPoint]) / 2
        midline[iPoint,1] = (yHP[iPoint] + yLP[iPoint]) / 2
        midline[iPoint,2] = (zHP[iPoint] + zLP[iPoint]) / 2

    return midline


def makeCrossSectionLayerAreas_perimeter(iStation,stationStacks,params,thicknessScaling,lpHpside,isFlatback,TEangle,lastRoundStation,partNameID, nModeledLayers,crossSectionNormal,lpHpCurveDict,materialIDsUsed):
    partName=lpHpside+'shell'

    #Assumes that #HP side is made first
    if lpHpside.lower() == 'hp':
        lpHpsideIndex=0
        camberOffsetSign=1
    else:
        lpHpsideIndex=1
        camberOffsetSign=-1

    offsetSign_camberID=printOffsetDirectionCheck(lpHpCurveDict['camberID'],'LP',crossSectionNormal)
    
    baseCurveIndexCT=0
    stationStacks.shape
    nStationLayups=len(stationStacks)
    stationStacks.shape

    lastPerimeter=nStationLayups-2
    for iPerimeter in range(nStationLayups-1): #Skip the last stack since the current and the next stack are generated at the same time.

        print(f'NEW Perimeter, iPerimeter={iPerimeter}')
        currentStack = stationStacks[iPerimeter]
        nextStack = stationStacks[iPerimeter+1]

        currentStackLayerThicknesses=np.array(currentStack.layerThicknesses())/1000
        nextStackLayerThicknesses=np.array(nextStack.layerThicknesses())/1000
        print(iPerimeter)

        cubit.cmd(f'curve {lpHpCurveDict["baseCurveIDs"][lpHpsideIndex][baseCurveIndexCT]} copy')
        currentBaseCurveID=get_last_id("curve")
        baseCurveIndexCT+=1
        cubit.cmd(f'curve {lpHpCurveDict["baseCurveIDs"][lpHpsideIndex][baseCurveIndexCT]} copy')
        nextBaseCurveID=get_last_id("curve")
        baseCurveIndexCT+=1

        currentStackSurfaceList=[]
        transitionStackSurfaceList=[]
        nextStackSurfaceList=[]

        #Find out if the next or the current stack contains reinforcement. The foam/balsa will be build arround the reinf.
        currentStackHasReinf = False
        nextStackHasReinf = False
        #Search for reinforcements
        for ply in currentStack.plygroups:
            if 'reinf' in ply.component.lower() or 'spar' in ply.component.lower():
                currentStackHasReinf=True
                break
        for ply in nextStack.plygroups:
            if 'reinf' in ply.component.lower() or 'spar' in ply.component.lower():
                nextStackHasReinf=True
                break

        currentStackLayerOffset = 0
        nextStackLayerOffset = 0
        layerThicknessTransitionLengths = []
        #Get all offsets and layerThicknessTransitionLengths
        thinestLayerThicknessCurrentStack = 1e+22 #initialize to a large value
        thinestLayerThicknessNextStack = 1e+22
        for iModeledLayers in range(nModeledLayers):
            currentStackLayerOffset+=currentStackLayerThicknesses[iModeledLayers]
            nextStackLayerOffset+=nextStackLayerThicknesses[iModeledLayers]

            adjacentLayerMissmatch=abs(currentStackLayerOffset-nextStackLayerOffset)


            if adjacentLayerMissmatch > params['minimumTransitionLength']:
                layerThicknessTransitionLengths.append(adjacentLayerMissmatch / tan(math.radians(params['transitionTaperAngle'])))
            else:
                layerThicknessTransitionLengths.append(params['minimumTransitionLength'])

            #Also find the thinest layer in stack for meshing purposes
            if currentStackLayerThicknesses[iModeledLayers]>0 and currentStackLayerThicknesses[iModeledLayers] < thinestLayerThicknessCurrentStack:
                thinestLayerThicknessCurrentStack=currentStackLayerThicknesses[iModeledLayers]

            if nextStackLayerThicknesses[iModeledLayers]>0 and nextStackLayerThicknesses[iModeledLayers] < thinestLayerThicknessNextStack:
                thinestLayerThicknessNextStack=nextStackLayerThicknesses[iModeledLayers] 

        maxLayerThicknessTransitionLength=max(layerThicknessTransitionLengths)



        if currentStackHasReinf and  not nextStackHasReinf:
            leftOffsetCurve=currentBaseCurveID
            cubit.cmd(f'split curve {nextBaseCurveID} distance {maxLayerThicknessTransitionLength} from start ')
            splitBaseCurve_ShortID=get_last_id("curve")-1
            rightOffsetCurve=get_last_id("curve")
            transitionStack = nextStack
        elif nextStackHasReinf  and not currentStackHasReinf:
            rightOffsetCurve=nextBaseCurveID
            cubit.cmd(f'split curve {currentBaseCurveID} distance {maxLayerThicknessTransitionLength} from end ')
            leftOffsetCurve=get_last_id("curve")-1
            splitBaseCurve_ShortID=get_last_id("curve")
            transitionStack = currentStack
        else:
            if iPerimeter == 1:
                rightOffsetCurve=nextBaseCurveID
                cubit.cmd(f'split curve {currentBaseCurveID} distance {maxLayerThicknessTransitionLength} from end ')
                leftOffsetCurve=get_last_id("curve")-1
                splitBaseCurve_ShortID=get_last_id("curve")
                transitionStack = currentStack
            elif iPerimeter == 2:
                leftOffsetCurve=currentBaseCurveID
                cubit.cmd(f'split curve {nextBaseCurveID} distance {maxLayerThicknessTransitionLength} from start ')
                splitBaseCurve_ShortID=get_last_id("curve")-1
                rightOffsetCurve=get_last_id("curve")
                transitionStack = nextStack
            else:
                cubit.cmd(f'curve {currentBaseCurveID} copy')
                leftOffsetCurve=get_last_id("curve")
                cubit.cmd(f'split curve {currentBaseCurveID} distance {maxLayerThicknessTransitionLength} from end ')
                leftOffsetCurve=get_last_id("curve")-1
                leftTransitionCurve=get_last_id("curve")
                cubit.cmd(f'split curve {nextBaseCurveID} distance {maxLayerThicknessTransitionLength} from start ')
                rightTransitionCurve=get_last_id("curve")-1
                rightOffsetCurve=get_last_id("curve")
                cubit.cmd(f'create curve combine curve {leftTransitionCurve} {rightTransitionCurve}')
                splitBaseCurve_ShortID=get_last_id("curve")
                transitionStack = currentStack

        bottomLeftVertexCurveLeft,bottomRightVertexCurveLeft=selCurveVerts(leftOffsetCurve)
        bottomLeftVertexCurveRight,bottomRightVertexCurveRight=selCurveVerts(rightOffsetCurve)

        offsetSign_leftOffsetCurve=printOffsetDirectionCheck(leftOffsetCurve,lpHpside,crossSectionNormal)
        offsetSign_rightOffsetCurve=printOffsetDirectionCheck(rightOffsetCurve,lpHpside,crossSectionNormal)


        leftBottomCurve=leftOffsetCurve
        transitionBottomCurve=splitBaseCurve_ShortID
        rightBottomCurve=rightOffsetCurve



        #This if statement prepares all layer curves such that they taper at the TE
        if iPerimeter==0:
            currentStackRightCurves=[]
            currentStackLeftCurves=[]

            #Base curve copy
            cubit.cmd(f'curve {leftOffsetCurve} copy')
            baseCurveIDCopy=get_last_id("curve")
            offsetSign_baseCurveIDCopy=printOffsetDirectionCheck(baseCurveIDCopy,lpHpside,crossSectionNormal)
            
            #offset camber to make gap
            cubit.cmd(f'create curve offset curve {lpHpCurveDict["camberID"]} distance {camberOffsetSign*offsetSign_camberID*params["TE_adhesive"]/2} extended')
            camberOffset=get_last_id("curve")
            offsetSign_camberOffset=printOffsetDirectionCheck(camberOffset,lpHpside,crossSectionNormal)

            #Top Bounding Curve
            offsetDistance=1*offsetSign_baseCurveIDCopy*sum(currentStackLayerThicknesses)
            offsetCurveAndCombineFragmentsIfNeeded(baseCurveIDCopy,offsetDistance)
            topBoundingCurve=get_last_id("curve")
   

            if TEangle > 120:
                lpHpCurveDict['flatBackCurveID']=camberOffset

                curveStartOrEnd='start'
                extensionLength=1*cubit.curve(baseCurveIDCopy).length()
                baseCurveIDCopy=extendCurveAtVertexToLength(baseCurveIDCopy,extensionLength,curveStartOrEnd)

                curveStartOrEnd='start'
                extensionLength=1*cubit.curve(topBoundingCurve).length()
                topBoundingCurve=extendCurveAtVertexToLength(topBoundingCurve,extensionLength,curveStartOrEnd)  

                _,v1=selCurveVerts(baseCurveIDCopy)
                cubit.cmd(f'trim curve {baseCurveIDCopy} atintersection curve {lpHpCurveDict["flatBackCurveID"]} keepside vertex {v1}')
                baseCurveIDCopy=get_last_id("curve")
            else:

                curveStartOrEnd='start'
                extensionLength=1*cubit.curve(topBoundingCurve).length()
                topBoundingCurve=extendCurveAtVertexToLength(topBoundingCurve,extensionLength,curveStartOrEnd)
                keepCurve=2
                topBoundingCurve=streamlineCurveIntersections(camberOffset,topBoundingCurve,keepCurve)

            #Trim curve at TE.adhesive
            _,v1=selCurveVerts(topBoundingCurve)
            cubit.cmd(f'trim curve {topBoundingCurve} atintersection curve {lpHpCurveDict["flatBackCurveID"]} keepside vertex {v1}')
            topBoundingCurve=get_last_id("curve")
            offsetSign_topBoundingCurve=printOffsetDirectionCheck(topBoundingCurve,lpHpside,crossSectionNormal)

            #Make list of curves that will be used to taper each layer
            npts=30
            nStart=get_last_id("curve")+1
            for iPoint in range(npts):
                if iPoint==0:
                    cubit.cmd(f'create vertex on curve {baseCurveIDCopy} start')
                    cubit.cmd(f'create vertex on curve {topBoundingCurve} start')
                else:
                    cubit.cmd(f'create vertex on curve {baseCurveIDCopy} fraction {(iPoint)/(npts-1)} from start')
                    cubit.cmd(f'create vertex on curve {topBoundingCurve} fraction {(iPoint)/(npts-1)} from start')
                cubit.cmd(f'create curve vertex {get_last_id("vertex")-1} {get_last_id("vertex")}')
            nEnd=get_last_id("curve")
            curveIDs=list(range(nStart,nEnd+1))

            #TE Adhesive curve
            if isFlatback:
                cubit.cmd(f'curve {topBoundingCurve} copy')
                cubit.cmd(f'split curve {get_last_id("curve")} distance {params["TE_adhesive_width"]} from start')
                #HPTEadhesiveCurveID=get_last_id("curve")-1
                lpHpCurveDict['TEadhesiveCurveList'][lpHpsideIndex].append(get_last_id("curve")-1)
                cubit.cmd(f'delete curve {get_last_id("curve")}')
            else:
                v1,_=selCurveVerts(baseCurveIDCopy)
                v2,_=selCurveVerts(topBoundingCurve)
                cubit.cmd(f'split curve {lpHpCurveDict["flatBackCurveID"]} at vertex {v1} {v2}')
                cubit.cmd(f'delete curve {get_last_id("curve")} {get_last_id("curve")-2}')
                lpHpCurveDict['flatBackCurveID']=get_last_id("curve")-1
                cubit.cmd(f'curve {lpHpCurveDict["flatBackCurveID"]} copy')
                #HPTEadhesiveCurveID=get_last_id("curve")

            iLayer=0
            offsetDistance=1*offsetSign_baseCurveIDCopy*currentStackLayerThicknesses[iLayer]
            offsetCurveAndCombineFragmentsIfNeeded(baseCurveIDCopy,offsetDistance)
            firstLayerOffset=get_last_id("curve")

            curveStartOrEnd='start'
            firstLayerOffset=extendCurvePastCurveAndTrim(firstLayerOffset,curveStartOrEnd,lpHpCurveDict['flatBackCurveID'])

            #Only do the following if all layer thicknesses are unequal
            if abs(min(currentStack.layerThicknesses())-max(currentStack.layerThicknesses())) > 0.0001: 
                nStart=get_last_id("vertex")+1
                curveFraction=1.0/3
                for iCurve in curveIDs:
                    cubit.cmd(f'create vertex on curve {iCurve} fraction {curveFraction} from start')
                nEnd=get_last_id("vertex")
                vertexList=list(range(nStart,nEnd+1))
                cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
                adjustmentCurve=get_last_id("curve")
                cubit.cmd(f'delete vertex {l2s(vertexList[1:-1])}')

                keepCurve=2
                firstLayerOffset=streamlineCurveIntersections(adjustmentCurve,firstLayerOffset,keepCurve)

            cubit.cmd(f'create curve offset curve {lpHpCurveDict["camberID"]} distance {camberOffsetSign*offsetSign_camberID*params["TE_adhesive"]/2} extended')
            camberOffset=get_last_id("curve")

            offsetSign_topBoundingCurve=printOffsetDirectionCheck(topBoundingCurve,lpHpside,crossSectionNormal)

            offsetDistance=-1*offsetSign_topBoundingCurve*currentStackLayerThicknesses[-1]
            offsetCurveAndCombineFragmentsIfNeeded(topBoundingCurve,offsetDistance)
            lastLayerOffset=get_last_id("curve")


            curveStartOrEnd='start'
            lastLayerOffset=extendCurvePastCurveAndTrim(lastLayerOffset,curveStartOrEnd,lpHpCurveDict['flatBackCurveID'])

            if abs(min(currentStack.layerThicknesses())-max(currentStack.layerThicknesses())) > 0.0001: 
                nStart=get_last_id("vertex")+1
                curveFraction=2.0/3
                for iCurve in curveIDs:
                    cubit.cmd(f'create vertex on curve {iCurve} fraction {curveFraction} from start')
                nEnd=get_last_id("vertex")
                vertexList=list(range(nStart,nEnd+1))
                cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
                adjustmentCurve=get_last_id("curve")
                cubit.cmd(f'delete vertex {l2s(vertexList[1:-1])}')


                keepCurve=2
                lastLayerOffset=streamlineCurveIntersections(adjustmentCurve,lastLayerOffset,keepCurve)

                cubit.cmd(f'delete curve {l2s(curveIDs)}')

            cubit.cmd(f'split curve {firstLayerOffset} distance {params["TE_adhesive_width"]} from start')
            currentStackLeftCurves.append(get_last_id("curve")-1)
            currentStackRightCurves.append(get_last_id("curve"))
            curveLen=cubit.curve(currentStackLeftCurves[0]).length()
            cubit.cmd(f'split curve {baseCurveIDCopy} distance {curveLen} from start')
            currentStackLeftCurves.insert(0,get_last_id("curve")-1)
            currentStackRightCurves.insert(0,get_last_id("curve"))
            cubit.cmd(f'split curve {lastLayerOffset} distance {curveLen} from start')
            currentStackLeftCurves.append(get_last_id("curve")-1)
            currentStackRightCurves.append(get_last_id("curve"))
            cubit.cmd(f'split curve {topBoundingCurve} distance {curveLen} from start')
            currentStackLeftCurves.append(get_last_id("curve")-1)
            currentStackRightCurves.append(get_last_id("curve"))

#             if TEangle > 120:
#                 vertexList=[]
#                 for icurve in currentStackLeftCurves:
#                     v1,_=selCurveVerts(icurve)
#                     vertexList.append(v1)
#                 #cubit.cmd(f'delete curve {HPTEadhesiveCurveID}')
#                 cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
#                 #HPTEadhesiveCurveID=get_last_id("curve")
#                 lpHpCurveDict['TEadhesiveCurveList'][lpHpsideIndex].append(get_last_id("curve"))
#                 #HPTEadhesiveCurveList.append(HPTEadhesiveCurveID)


            #### Next Stack (the panel might intersect the camberline so the following is needed 
            nextStackCurves=[]
            cubit.cmd(f'curve {rightOffsetCurve} copy')
            baseCurveIDCopy=get_last_id("curve")
            offsetSign_baseCurveIDCopy=printOffsetDirectionCheck(baseCurveIDCopy,lpHpside,crossSectionNormal)
            nextStackCurves.append(baseCurveIDCopy)

            ### Offset camber to make gap
            #Offset is increased to create a larger clearance between HP LP shells so that the panels
            #to not self intersect during a simulation (this may not be needed)
            cubit.cmd(f'create curve offset curve {lpHpCurveDict["camberID"]} distance {camberOffsetSign*offsetSign_camberID*params["TE_adhesive"]*4} extended')
            camberOffset=get_last_id("curve")
            offsetSign_camberOffset=printOffsetDirectionCheck(camberOffset,lpHpside,crossSectionNormal)

            iLayer=0
            offsetDistance=1*offsetSign_baseCurveIDCopy*nextStackLayerThicknesses[0]
            offsetCurveAndCombineFragmentsIfNeeded(baseCurveIDCopy,offsetDistance)
            nextStackCurves.append(get_last_id("curve"))


            offsetDistance=1*offsetSign_baseCurveIDCopy*sum(nextStackLayerThicknesses)
            offsetCurveAndCombineFragmentsIfNeeded(baseCurveIDCopy,offsetDistance)
            topBoundingCurve=get_last_id("curve")

            keepCurve=2
            topBoundingCurve=streamlineCurveIntersections(camberOffset,topBoundingCurve,keepCurve)

            v1,_=selCurveVerts(baseCurveIDCopy)
            cubit.cmd(f'split curve {topBoundingCurve} at vertex {v1}')
            topBoundingCurve=get_last_id("curve")
            cubit.cmd(f'delete curve {get_last_id("curve")-1}')
            nextStackCurves.append(topBoundingCurve)

            offsetSign_topBoundingCurve=printOffsetDirectionCheck(topBoundingCurve,lpHpside,crossSectionNormal)


            offsetDistance=-1*offsetSign_topBoundingCurve*nextStackLayerThicknesses[-1]
            offsetCurveAndCombineFragmentsIfNeeded(topBoundingCurve,offsetDistance)
            lastLayerOffset=get_last_id("curve")
            nextStackCurves.insert(-1,lastLayerOffset)



        currentStackOffset=0
        nextStackOffset=0

        for iModeledLayers in range(nModeledLayers):
            currentStackOffset+=currentStackLayerThicknesses[iModeledLayers]
            nextStackOffset+=nextStackLayerThicknesses[iModeledLayers]

            #Special Treatment for TE
            if iPerimeter==0:
                #Create Left Areas Only

                if iModeledLayers==1:
                    materialID=currentStack.plygroups[0].materialid
                    plyAngle=currentStack.plygroups[0].angle
                else:
                    materialID=currentStack.plygroups[iModeledLayers].materialid
                    plyAngle=currentStack.plygroups[iModeledLayers].angle

                partNameID,materialIDsUsed=makeCrossSectionSurface(partName,currentStackLeftCurves[iModeledLayers+1],currentStackLeftCurves[iModeledLayers],materialID,plyAngle,partNameID,iModeledLayers,materialIDsUsed)
                
                if TEangle > 120:
                    lpHpCurveDict['TEadhesiveCurveList'][lpHpsideIndex].append(surfaceDict[get_last_id('surface')]['curves'][-1])
                        
                if iStation== lastRoundStation-1:
                    v1=surfaceDict[get_last_id("surface")]['verts'][0]
                    cubit.cmd(f'vertex {v1} rename "linear"')
                    v1=surfaceDict[get_last_id("surface")]['verts'][-1]
                    cubit.cmd(f'vertex {v1} rename "linear"')

                leftBottomCurve=currentStackRightCurves[iModeledLayers]
                leftTopCurve=currentStackRightCurves[iModeledLayers+1]
                [bottomLeftVertexCurveLeft,bottomRightVertexCurveLeft]=selCurveVerts(currentStackRightCurves[iModeledLayers])
                [topLeftVertexCurveLeft,topRightVertexCurveLeft]=selCurveVerts(currentStackRightCurves[iModeledLayers+1])

                #Create Left Areas Only
                rightBottomCurve=nextStackCurves[iModeledLayers]
                rightTopCurve=nextStackCurves[iModeledLayers+1]
                [bottomLeftVertexCurveRight,bottomRightVertexCurveRight]=selCurveVerts(nextStackCurves[iModeledLayers])
                [topLeftVertexCurveRight,topRightVertexCurveRight]=selCurveVerts(nextStackCurves[iModeledLayers+1])
            else:
                cubit.cmd(f'create curve offset curve {leftOffsetCurve} distance {offsetSign_leftOffsetCurve*currentStackOffset} extended')
                leftTopCurve=get_last_id("curve")
                [topLeftVertexCurveLeft,topRightVertexCurveLeft]=selCurveVerts(get_last_id("curve"))
                cubit.cmd(f'create curve offset curve {rightOffsetCurve} distance {offsetSign_rightOffsetCurve*nextStackOffset} extended')
                lastOffsetCurve=get_last_id("curve")
                rightTopCurve=get_last_id("curve")
                [topLeftVertexCurveRight,topRightVertexCurveRight]=selCurveVerts(get_last_id("curve"))


            if iPerimeter==lastPerimeter:
                curveStartOrEnd='end'
                lastOffsetCurve=extendCurvePastCurveAndTrim(lastOffsetCurve,curveStartOrEnd,lpHpCurveDict['LEadhesiveCurveIDs'][lpHpsideIndex][1])
                rightTopCurve=lastOffsetCurve
                [bottomLeftVertexCurveRight,bottomRightVertexCurveRight]=selCurveVerts(rightBottomCurve)
                [topLeftVertexCurveRight,topRightVertexCurveRight]=selCurveVerts(rightTopCurve)



            cubit.cmd(f'create curve vertex {topRightVertexCurveLeft} {topLeftVertexCurveRight}')
            transitionTopCurve=get_last_id("curve")
            cubit.cmd(f'create curve vertex {bottomLeftVertexCurveLeft} {topLeftVertexCurveLeft}')
            cubit.cmd(f'create curve vertex {bottomRightVertexCurveLeft} {topRightVertexCurveLeft}')
            nCurvesFinal=get_last_id("curve")
            leftSideCurves = [i for i in range(nCurvesFinal-1,nCurvesFinal+1)]
            cubit.cmd(f'create curve vertex {bottomLeftVertexCurveRight} {topLeftVertexCurveRight}')
            cubit.cmd(f'create curve vertex {bottomRightVertexCurveRight} {topRightVertexCurveRight}')
            nCurvesFinal=get_last_id("curve")
            rightSideCurves = [i for i in range(nCurvesFinal-1,nCurvesFinal+1)]

            if iPerimeter==lastPerimeter: 
                if isFlatback:
                    cubit.cmd(f'split curve {lpHpCurveDict["LEadhesiveCurveIDs"][lpHpsideIndex][1]} at vertex {topRightVertexCurveRight}')
                    rightSideCurves[-1]=get_last_id("curve")-1
                    lpHpCurveDict['LEadhesiveCurveIDs'][lpHpsideIndex][1]=get_last_id("curve")
                lpHpCurveDict['LEadhesiveCurveList'][lpHpsideIndex].append(rightSideCurves[-1])


            ### Create Surfaces ###
            #Sufaces for currentStack
            materialID=currentStack.plygroups[iModeledLayers].materialid
            plyAngle=currentStack.plygroups[iModeledLayers].angle
            partNameID,materialIDsUsed=makeCrossSectionSurface(partName,leftTopCurve,leftBottomCurve,materialID,plyAngle,partNameID,iModeledLayers,materialIDsUsed)
            currentStackSurfaceList.append(get_last_id("surface"))

            #Surfaces for transitionStack
            materialID=transitionStack.plygroups[iModeledLayers].materialid
            plyAngle=transitionStack.plygroups[iModeledLayers].angle
            partNameID,materialIDsUsed=makeCrossSectionSurface(partName,transitionTopCurve,transitionBottomCurve,materialID,plyAngle,partNameID,iModeledLayers,materialIDsUsed)
            transitionStackSurfaceList.append(get_last_id("surface"))

            #Surfaces for nextStack
            materialID=nextStack.plygroups[iModeledLayers].materialid
            plyAngle=nextStack.plygroups[iModeledLayers].angle
            partNameID,materialIDsUsed=makeCrossSectionSurface(partName,rightTopCurve,rightBottomCurve,materialID,plyAngle,partNameID,iModeledLayers,materialIDsUsed)
            nextStackSurfaceList.append(get_last_id("surface"))

            ### Reset ###
            #Reset curves
            leftBottomCurve=leftTopCurve
            transitionBottomCurve=transitionTopCurve
            rightBottomCurve=rightTopCurve

            #Reset vertices
            bottomLeftVertexCurveLeft=topLeftVertexCurveLeft
            bottomRightVertexCurveLeft=topRightVertexCurveLeft
            bottomLeftVertexCurveRight=topLeftVertexCurveRight
            bottomRightVertexCurveRight=topRightVertexCurveRight


        #Build spar caps
        if iPerimeter==1:
            lpHpCurveDict['webInterfaceCurves'][lpHpsideIndex]=[rightTopCurve]
            for ic, currentCurveID in enumerate(lpHpCurveDict['sparCapBaseCurves'][lpHpsideIndex]):
                bottomCurve=currentCurveID
                offSetSign=printOffsetDirectionCheck(bottomCurve,lpHpside,crossSectionNormal)

                for it, thickness in enumerate(nextStackLayerThicknesses):
                    cubit.cmd(f'create curve offset curve {bottomCurve} distance {offSetSign*thickness} extended')
                    topCurve=get_last_id("curve")

                    materialID=nextStack.plygroups[it].materialid
                    plyAngle=nextStack.plygroups[it].angle
                    partNameID,materialIDsUsed=makeCrossSectionSurface(partName,topCurve,bottomCurve,materialID,plyAngle,partNameID,it,materialIDsUsed)
                    nextStackSurfaceList.append(get_last_id("surface"))     

                    if it==2 and ic!=3:
                        lpHpCurveDict['webInterfaceCurves'][lpHpsideIndex].append(topCurve)
                    bottomCurve=topCurve
        elif iPerimeter==2:
            lpHpCurveDict['webInterfaceCurves'][lpHpsideIndex].append(leftTopCurve)
    return partNameID,lpHpCurveDict
####################################################
####################################################
####################################################
####################################################
####################################################


def createSimplistSurfaceForTEorLEadhesive(partName,adhesiveCurveList,adhesiveMatID,partNameID,nModeledLayers,materialIDsUsed):
    for iCurve in range(len(adhesiveCurveList[0])):
        plyAngle=0 #Ply angle is always zero since adhesive is always assumed as isotropic
        partNameID,materialIDsUsed=makeCrossSectionSurface(partName,adhesiveCurveList[1][iCurve],adhesiveCurveList[0][iCurve],crosssectionParams['adhesiveMatID'],plyAngle,partNameID,nModeledLayers+1,materialIDsUsed)
        
    return partNameID
    
def printSineCurveBetweenTwoVerts(vBot,vTop,amplitude,direction):
    nSineCurveSamplePoints=7
    cubit.cmd(f'create curve vertex {vBot} {vTop}')
    
    idCurve=get_last_id("curve")

    if round(amplitude,3)>0:
        nStart=get_last_id("vertex")+1
        cubit.cmd(f'create vertex on curve {get_last_id("curve")} segment {nSineCurveSamplePoints-1}')
        nEnd=get_last_id("vertex")
        sineCurveSamplePoints=np.linspace(0,pi,nSineCurveSamplePoints)
        vertexOffsets=amplitude*np.sin(sineCurveSamplePoints)
        vertexList=list(range(nStart,nEnd+1))
        for iVert, vertexOffset in enumerate(vertexOffsets[1:-1]): #skip first and last point since those are considered fixed and the offset is zero anyway
            cubit.cmd(f'move vertex {vertexList[iVert]} {direction} {vertexOffset}')
        vertexList.insert(0,vBot)
        vertexList.append(vTop)
        cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
        cubit.cmd(f'delete curve {idCurve}')
    return get_last_id("curve")

def makeCrossSectionLayerAreas_web(aftWebStack,foreWebStack,webInterfaceCurves,crosssectionParams,partNameID,crossSectionNormal,nModeledLayers,materialIDsUsed):
    aftWebOverwrapThickness=(aftWebStack.layerThicknesses()[0]+aftWebStack.layerThicknesses()[-1])/1000
    foreWebOverwrapThickness=(foreWebStack.layerThicknesses()[0]+foreWebStack.layerThicknesses()[-1])/1000
    partName='web'
    ### First create the first two layers. The first layer is the adhesive. The second layer is the web overwrap layer    
    for iCurveList,curveList in enumerate(webInterfaceCurves):
        nBaseCurvesWeb=len(curveList)
        if iCurveList == 0:
            lpHpside='HP'
        else:
            lpHpside='LP'
        for iCurve,bottomCurve in enumerate(curveList):
            offSetSign=printOffsetDirectionCheck(bottomCurve,lpHpside,crossSectionNormal) 


            if iCurve<nBaseCurvesWeb/2:
                layerThicknesses=[crosssectionParams['Web_aft_adhesive'],aftWebOverwrapThickness]
            else:
                layerThicknesses=[crosssectionParams['Web_fore_adhesive'],foreWebOverwrapThickness,]
            


            for it, thickness in enumerate(layerThicknesses):
                cubit.cmd(f'create curve offset curve {bottomCurve} distance {offSetSign*thickness} extended')
                topCurve=get_last_id("curve")

                if it==0:
                    materialID=crosssectionParams['adhesiveMatID']
                    plyAngle=0
                else:
                    if iCurve<nBaseCurvesWeb/2:
                        materialID=aftWebStack.plygroups[0].materialid
                        plyAngle=aftWebStack.plygroups[0].angle
                    else:
                        materialID=foreWebStack.plygroups[0].materialid
                        plyAngle=foreWebStack.plygroups[0].angle

                partNameID,materialIDsUsed=makeCrossSectionSurface(partName, topCurve,bottomCurve,materialID,plyAngle,partNameID,nModeledLayers+it,materialIDsUsed)

                bottomCurve=topCurve

            #update web interface curves for vertical web retions
            webInterfaceCurves[iCurveList][iCurve]=topCurve

    ### Create vertical web regions  

    #remove curves that are not going to be part of the vertical web
    for iCurveList,curveList in enumerate(webInterfaceCurves):
        curveList.pop(3)
        curveList.pop(3)


    nBaseCurvesWeb=len(webInterfaceCurves[0])
    for iCurve in range(nBaseCurvesWeb):

        vHP,_=selCurveVerts(webInterfaceCurves[0][iCurve])
        vLP,_=selCurveVerts(webInterfaceCurves[1][iCurve])
        topCurve=printSineCurveBetweenTwoVerts(vHP,vLP,crosssectionParams['maxWebImperfectionDistance'],'x')
        _,vHP=selCurveVerts(webInterfaceCurves[0][iCurve])
        _,vLP=selCurveVerts(webInterfaceCurves[1][iCurve])
        bottomCurve=printSineCurveBetweenTwoVerts(vHP,vLP,crosssectionParams['maxWebImperfectionDistance'],'x')


        if iCurve<nBaseCurvesWeb/2:
            materialID=aftWebStack.plygroups[iCurve].materialid
            plyAngle=aftWebStack.plygroups[iCurve].angle
        else:
            materialID=foreWebStack.plygroups[iCurve-int(nBaseCurvesWeb/2)].materialid
            plyAngle=foreWebStack.plygroups[iCurve-int(nBaseCurvesWeb/2)].angle
        partNameID,materialIDsUsed=makeCrossSectionSurface(partName, topCurve,bottomCurve,materialID,plyAngle,partNameID,nModeledLayers+it+2+iCurve,materialIDsUsed)
    return partNameID, (vHP,vLP)

def writeCubitCrossSection(iStation,blade,hasWebs,aftWebStack,foreWebStack,iLE,crosssectionParams,geometryScaling,thicknessScaling,isFlatback,lastRoundStation,materialIDsUsed):
    print(f'Writing Station: {iStation}')
    partNameID=0
    webBaseCurves=[]
    webCornerVerts=[]
    crossSectionNormal=getCrossSectionNormalVector(np.array([blade.keypoints[2,:,iStation],blade.keypoints[3,:,iStation],blade.keypoints[7,:,iStation]]))


    #### Step one create outer mold line
    xyz = getBladeGeometryForStation(blade,iStation) * geometryScaling

    splinePoints=xyz[1:iLE,:]
    writeSplineFromCoordinatePoints(splinePoints)
    hpKeyCurve=get_last_id("curve")

    xyz=np.flip(xyz,0);
    splinePoints=xyz[1:iLE,:]
    writeSplineFromCoordinatePoints(splinePoints)
    lpKeyCurve=get_last_id("curve")


    flatBack_vBot,_=selCurveVerts(hpKeyCurve)
    flatBack_vTop,_=selCurveVerts(lpKeyCurve)

    firstPoint = xyz[-2,:]
    secondPont = xyz[1,:]
    flatBackLength = np.linalg.norm(secondPont - firstPoint)

    TEangle = getTEangle(xyz)
    print(f'station {iStation}')
    print(f'edgeLength={flatBackLength*1000}')
    print(crosssectionParams)
    print(f'athickness={crosssectionParams["TE_adhesive"]*1000}') 
    print(f'TE_adhesive_width {crosssectionParams["TE_adhesive_width"]*1000}')
    print(f'TEangle {TEangle}')
    flatBackCurve=cubit.create_curve(cubit.vertex(flatBack_vBot),cubit.vertex(flatBack_vTop))
    flatBackCurveID=flatBackCurve.id()

    #### Extend flatback ###
    curveStartOrEnd='start'
    extensionLength=4*cubit.curve(flatBackCurveID).length()
    flatBackCurveID=extendCurveAtVertexToLength(flatBackCurveID,extensionLength,curveStartOrEnd)

    curveStartOrEnd='end'
    extensionLength=0.5*cubit.curve(flatBackCurveID).length()
    flatBackCurveID=extendCurveAtVertexToLength(flatBackCurveID,extensionLength,curveStartOrEnd)

    #remove beginning portion of curve due to sometimes high curvature (bad geometry fix)
    hpKeyCurve=removeBadTEgeometry(hpKeyCurve,flatBackCurveID)
    lpKeyCurve=removeBadTEgeometry(lpKeyCurve,flatBackCurveID)

    adhesiveThickness=crosssectionParams['LE_adhesive']
    nStacks=len(blade.stacks)

    LEHPstackThickness=sum(blade.stacks[int(nStacks/2.)-1,iStation].layerThicknesses())/1000
    LELPstackThickness=sum(blade.stacks[int(nStacks/2.),iStation].layerThicknesses())/1000

    #Define variables with HP side in index 0, LP side in inex 1
    
    lpHpCurveDict={}
    lpHpCurveDict['sparCapBaseCurves']=[[],[]] 
    lpHpCurveDict['webInterfaceCurves']=[[],[]]
    lpHpCurveDict['baseCurveIDs']=[[],[]]
    lpHpCurveDict['TEadhesiveCurveList']=[[],[]]
    lpHpCurveDict['LEadhesiveCurveList']=[[],[]]

    hpKeyCurve,lpKeyCurve,lpHpCurveDict['LEadhesiveCurveIDs']=writeLEAdhesiveCurves(LEHPstackThickness,LELPstackThickness,adhesiveThickness,hpKeyCurve,lpKeyCurve,crossSectionNormal)


    keyCurves=splitCurveAtCoordintePoints(blade.keypoints[1:5,:,iStation],hpKeyCurve)
    lpHpCurveDict['baseCurveIDs'][0],lpHpCurveDict['sparCapBaseCurves'][0]=splitKeyCurves(keyCurves)


    temp=np.flip(blade.keypoints[:,:,iStation],0)
    keyCurves=splitCurveAtCoordintePoints(temp[1:5,:],lpKeyCurve)
    lpHpCurveDict['baseCurveIDs'][1],lpHpCurveDict['sparCapBaseCurves'][1]=splitKeyCurves(keyCurves)

    xyz=getMidLine(iLE)
    npts,_=xyz.shape
    npts=round(npts*0.75) #Only need write first 3/4 of camber line since LE is constructed another way

    splinePoints=xyz[0:npts,:]
    writeSplineFromCoordinatePoints(splinePoints)
    camberID=get_last_id("curve")

    #Extend
    curveStartOrEnd='start'
    extensionLength=0.5*cubit.curve(camberID).length()
    camberID=extendCurveAtVertexToLength(camberID,extensionLength,curveStartOrEnd)

    lpHpCurveDict['camberID']=camberID
    lpHpCurveDict['flatBackCurveID']=flatBackCurveID


    nModeledLayers=3

    lpHpside='HP'
    
    
    partNameID,lpHpCurveDict=makeCrossSectionLayerAreas_perimeter(iStation,blade.stacks[1:6,iStation],crosssectionParams,thicknessScaling,lpHpside,isFlatback,TEangle,lastRoundStation,partNameID,nModeledLayers,crossSectionNormal,lpHpCurveDict,materialIDsUsed)



    lpHpside='LP'
    temp=blade.stacks[:,iStation]
    temp=np.flip(temp)
    partNameID,lpHpCurveDict=makeCrossSectionLayerAreas_perimeter(iStation,temp[1:6],crosssectionParams,thicknessScaling,lpHpside,isFlatback,TEangle,lastRoundStation,partNameID,nModeledLayers,crossSectionNormal,lpHpCurveDict,materialIDsUsed)

    partName='shell'
    partNameID=createSimplistSurfaceForTEorLEadhesive(partName,lpHpCurveDict['LEadhesiveCurveList'],crosssectionParams['adhesiveMatID'],partNameID,nModeledLayers,materialIDsUsed)
    
    

    partNameID=0 #Reset since outer areoshell is complete (LE adhesive is accouted for as aeroshell)
    if iStation>lastRoundStation:    
        partName='flatTEadhesive'
    else:
        partName='roundTEadhesive'
        
    partNameID=createSimplistSurfaceForTEorLEadhesive(partName,lpHpCurveDict['TEadhesiveCurveList'],crosssectionParams['adhesiveMatID'],partNameID,nModeledLayers,materialIDsUsed)



    birdsMouthVerts=[]
    if hasWebs:
        partNameID=0 #Reset since outer areoshell is complete (LE adhesive is accouted for as aeroshell)

        partNameID,birdsMouthVerts=makeCrossSectionLayerAreas_web(aftWebStack,foreWebStack,lpHpCurveDict['webInterfaceCurves'],crosssectionParams,partNameID,crossSectionNormal,nModeledLayers,materialIDsUsed)   
    
    


    parseString=f'with name "*station{iStation}*"'
    crossSectionalSurfaces=parse_cubit_list('surface', parseString)
    for surfaceID in crossSectionalSurfaces:
        n=get_surface_normal(surfaceID)
        if n[2]<0:
            cubit.cmd(f'reverse surface {surfaceID}')

    cubit.cmd(f'delete vertex all with Is_Free')

    return birdsMouthVerts

def verifyWebCuttingAmplitude(amplitude,tolerance):
    #Check to make sure that the amplitude does not result sharp volumes by cutting near a station location
    for iStationCheck in range(iStationFirstWeb+1,iStationLastWeb+1):
        bladeSegmentLength=blade.ispan[iStationCheck]-blade.ispan[iStationFirstWeb]
        gap=bladeSegmentLength-amplitude
        #print(f'bladeSegmentLength: {bladeSegmentLength}\ngap {gap}')

        if abs(gap) > tolerance:
            break
        else:
            if gap > 0:
                amplitude=bladeSegmentLength-tolerance
            else:
                amplitude=bladeSegmentLength+tolerance
            break
    #print(f'new amplitude {amplitude} \nnew gap = {bladeSegmentLength-amplitude}')
    return amplitude

def makeBirdsMouth(birdsMouthVerts,amplitudeFraction):
    ### Make birds mouth volume that will cut the web volumes ###
    #############################################################
    
    #This function must be ran before merging the volumes since "birdsMouthVerts" will change during mergeing

    v1=cubit.vertex(birdsMouthVerts[0])
    v2=cubit.vertex(birdsMouthVerts[1])
    distance=getDist(v1.coordinates(),v2.coordinates())[0] 
    create_curve(v1,v2)

    #Make the birds mouth cut-out start 5% from where the web meets the aeroshell
    cubit.cmd(f'create vertex on curve {get_last_id("curve")}  distance {0.05*distance} from start')
    cubit.cmd(f'create vertex on curve {get_last_id("curve")}  distance {0.05*distance} from end')
    v1=cubit.vertex(get_last_id("vertex")-1)
    v2=cubit.vertex(get_last_id("vertex"))
    straightLine=create_curve(v1,v2)



    amplitude=amplitudeFraction*distance
    tolerance=distance*0.05

    amplitude=verifyWebCuttingAmplitude(amplitude,tolerance)



    curvedLine=cubit.curve(printSineCurveBetweenTwoVerts(v1.id(),v2.id(),amplitude,'z'))
    cubit.cmd(f'create surface skin curve {curvedLine.id()} {straightLine.id()}')
    baseSurface=get_last_id("surface")

    midPoint=list(curvedLine.position_from_fraction(0.5))
    tangent=straightLine.tangent(midPoint)

    #Get the cross-section normal 
    parseString=f'in surface with name "*webStation{iStationFirstWeb}*"'
    surfaceID=parse_cubit_list('surface', parseString)[0] #Pick the first surface in this list since all on same plane
    coords=cubit.get_center_point("surface", surfaceID)
    surfaceNormal=cubit.surface(surfaceID).normal_at(coords)
    cutBlockLength=5*max(blade.ichord)
    sweepDirection=np.array(vectNorm(crossProd(list(tangent),list(surfaceNormal))))

    cubit.cmd(f'sweep surface {baseSurface} direction {l2s(sweepDirection)} distance {cutBlockLength}')
    cubit.cmd(f'move volume {get_last_id("volume")} x {-cutBlockLength/2*sweepDirection[0]} y {-cutBlockLength/2*sweepDirection[1]} z {-cutBlockLength/2*sweepDirection[2]}')


    cuttingVolume=get_last_id("volume")


    parseString=f'with name "*webStation*"'
    webVolumes=parse_cubit_list('volume', parseString)

    cubit.cmd(f'subtract volume {cuttingVolume} from volume {l2s(webVolumes)}')

    return

def writeVABSinput():
    #Write VABS inputfile
    if crosssectionParams['elementShape'].lower() == 'quad':
        expandedConnectivityString='face'
    elif crosssectionParams['elementShape'].lower() == 'tri':
        expandedConnectivityString='tri'
    else:
        raise NameError(f'Element type: {crosssectionParams["elementShape"]} not supported' ) 

    parseString=f'with name "*surface*"'
    surfaceIDs=parse_cubit_list('surface', parseString)  


    ######Write VABS input file
    nnodes=get_node_count()
    nelem=get_element_count()
    nblocks=get_block_count()
    nmate=len(blade.materials)
    nlayer=len(surfaceIDs) #Onle VABS layer is defined for each surface



    with open(VABSfileNameBase+str(iStation)+'-t-0.in', 'w') as f:
        f.write(f'1 {nlayer}\n') #New format. One layer definiton for each element.
        f.write('1 0 0\n')
        f.write('0 0 0 0\n')
        f.write(f'{nnodes} {nelem} {nmate}\n')
        #Write Nodes
        for iNd in range(nnodes):
            nodeID=iNd+1
            coords=list(get_nodal_coordinates(nodeID))
            f.write(f'{nodeID} {coords[0]} {coords[1]}\n')
        f.write('\n\n')
        #Write Elements
        maxNumberOfPossibleNodes=9
        for iEl in range(nelem):
            elementID=iEl+1
            nodesIDs=cubit.get_expanded_connectivity(crosssectionParams['elementShape'],elementID)
            tempStr=str(nodesIDs) #convert tuple to string
            tempStr=tempStr.replace('(','')
            tempStr=tempStr.replace(')','')
            tempStr=tempStr.replace(',',' ')
            nZeros=maxNumberOfPossibleNodes-len(nodesIDs)
            tempStr2=str([0]*nZeros)
            tempStr2=tempStr2.replace('[','')
            tempStr2=tempStr2.replace(']','')
            tempStr2=tempStr2.replace(',',' ')
            f.write(f'{elementID} {tempStr} {tempStr2}\n') 
        #Write ply angle for all but the TE adhesive

        for iSurface, surfaceID in enumerate(surfaceIDs):
            for iEl,elementID in enumerate(get_surface_quads(surfaceID)):
                nodesIDs=cubit.get_expanded_connectivity('face',elementID)
                coords=[]
                for iNd,nodeID in enumerate(nodesIDs):
                    coords.append(list(get_nodal_coordinates(nodeID)))
                coords=np.array(coords)
                # #######For Plotting - find the larges element side length #######
                # distances=[]
                # for iNd,nodeID in enumerate(nodesIDs):
                    # for jNd,nodeIDj in enumerate(nodesIDs):
                        # distances.append(norm(vectSub(coords[iNd],coords[jNd])))
                # length=max(distances)
                # #######For Plotting - find the larges element side length #######
                coords=np.mean(coords,0)
                #coords=cubit.get_center_point(crosssectionParams['elementShape'], elementID)


    #                             minDist=inf #initialize
    #                             closestCurveID=nan #initialize
    #                             #Since there are possibly many curves due to the offset operation, see which curve is closeset to element center
    #                             for iCurve, curveID in enumerate(curves):
    #                                 temp=cubit.curve(curveID).closest_point(coords)
    #                                 distance=getDist(coords,temp)[0]
    #                                 if distance < minDist:
    #                                     minDist=distance
    #                                     closestCurveID=curveID

                curveIDforMatOri=cubit.surface(surfaceID).curves()[0]
                curveLocationForTangent=curveIDforMatOri.closest_point(coords)
                x=curveIDforMatOri.tangent(curveLocationForTangent)[0]
                y=curveIDforMatOri.tangent(curveLocationForTangent)[1]
                z=curveIDforMatOri.tangent(curveLocationForTangent)[2]
                tangentDirection=vectNorm([x,y,z])  #Unit vector of tangent
                theta1=math.atan2(tangentDirection[1],tangentDirection[0])*180/pi
                f.write(f'{elementID} {iSurface+1} {theta1}\n')
                # #######Only needed For Plotting Orientation Check#######
                # cubit.create_vertex(coords[0],coords[1],coords[2])
                # iVert1=get_last_id("vertex")
                # cubit.create_vertex(coords[0]+length*tangentDirection[0],coords[1]+length*tangentDirection[1],coords[2]+length*tangentDirection[2])
                # iVert2=get_last_id("vertex")
                # cubit.cmd(f'create curve vertex {iVert1} {iVert2}')
                # #######Only needed For Plotting Orientation Check#######
            ####Normal to curve
                # axialDirection=crossSectionNormal #There will be a slight error here for highly tapeded regions
                # normalDirection=crossProd(axialDirection,tangentDirection)
                # #######Only needed For Plotting Orientation Check#######
                # cubit.create_vertex(coords[0]+length*normalDirection[0],coords[1]+length*normalDirection[1],coords[2]+length*normalDirection[2])
                # c
                # cubit.cmd(f'create curve vertex {iVert1} {iVert2}')
                # #######Only needed For Plotting Orientation Check#######
        #Define Plies
        for iSurface, surfaceID in enumerate(surfaceIDs):
            VABSmaterialID=surfaceDict[surfaceID]['materialID']+1
            plyAngle=surfaceDict[surfaceID]['plyAngle']
            f.write(f'{iSurface+1} {VABSmaterialID} {plyAngle}\n') 
        #Define Materials
        for imat in range(len(blade.materials)):
            VABSmaterialID=imat+1
            material=blade.materials[imat]
            f.write(f'{VABSmaterialID} {1} \n')
            f.write(f'{material.ex} {material.ey} {material.ez}\n')
            f.write(f'{material.gxy} {material.gxz} {material.gyz}\n')
            f.write(f'{material.prxy} {material.prxz} {material.pryz}\n')
            f.write(f'{material.density}\n')
    print('Done writing VABS input')
    return
#Main script fuctions
def getLastRoundStation(blade,TEangleLimit): 
    lastRoundStation = 0
    for iStation in range(len(blade.ispan)):
        xyz = getBladeGeometryForStation(blade,iStation)
        xyz = np.flip(xyz,0)
        TEangle=getTEangle(xyz)
        #fprintf('iStation #i, angle #f\n',iStation,TEangle)
        if TEangle < TEangleLimit:
            lastRoundStation = iStation -1
            break
    return lastRoundStation
def getTEangle(xyz):
    v1 = xyz[2,:] - xyz[1,:]
    v2 = xyz[-3,:] - xyz[-2,:]
    return math.degrees(math.acos(v1.dot(np.transpose(v2)) / (np.linalg.norm(v1) * np.linalg.norm(v2))))

# SED-like substitution
def copy_and_replace(fin, fout, replacements):
    inf = open(fin, 'r')
    outf = open(fout, 'w')
    for line in inf:
        for src, target in replacements.items():
            line = line.replace(src, target)
        outf.write(line)
    inf.close()
    outf.close()







TotalStations = np.asarray(blade.ispan).size
bladeMaterials = blade.materials.copy()      ##### COPYING NOT WORKING
nMaterials = np.asarray(bladeMaterials).size
iLE = blade.LEindex+1
if 'a' in myCase.lower():
    #Replace adhesive with triax
    iAdhesive = 1
    iOther = 4
    bladeMaterials[iAdhesive].type = bladeMaterials[iOther].type
    bladeMaterials[iAdhesive].ex = bladeMaterials[iOther].ex
    bladeMaterials[iAdhesive].ey = bladeMaterials[iOther].ey
    bladeMaterials[iAdhesive].ez = bladeMaterials[iOther].ez
    bladeMaterials[iAdhesive].gxy = bladeMaterials[iOther].gxy
    bladeMaterials[iAdhesive].gyz = bladeMaterials[iOther].gyz
    bladeMaterials[iAdhesive].gxz = bladeMaterials[iOther].gxz
    bladeMaterials[iAdhesive].prxy = bladeMaterials[iOther].prxy
    bladeMaterials[iAdhesive].pryz = bladeMaterials[iOther].pryz
    bladeMaterials[iAdhesive].prxz = bladeMaterials[iOther].prxz
    bladeMaterials[iAdhesive].density = bladeMaterials[iOther].density
elif 'b' in myCase.lower():
        pass

hasWebs = []
webNumber = 1
for iStation in range(len(blade.swstacks[webNumber])):
    if not len(blade.swstacks[webNumber][iStation].plygroups)==0 :
        hasWebs.append(True) 
    else:
        hasWebs.append(False)
        
#### WARNING - Last station never has webs. Fix later
hasWebs.append(False)     
#### WARNING - Last station never has webs. Fix later



#Create Referece line as a spline
temp=np.array([blade.sweep,blade.prebend,blade.ispan]).transpose()
writeSplineFromCoordinatePoints(temp)
spanwiseMatOriCurve=get_last_id("curve")



np.arange(iStationStart,iStationEnd+dStation,dStation).reshape(-1)



#crosssectionParams values that vary with spanwise location

for iStation in np.arange(iStationStart,iStationEnd+dStation,dStation).reshape(-1):
    #if 'a' in myCase.lower():
    #Adhesive thicknesses parameters
    allAdhesiveThickness = 1
    crosssectionParams['Web_fore_adhesive'] = allAdhesiveThickness * thicknessScaling
    crosssectionParams['Web_aft_adhesive'] = allAdhesiveThickness * thicknessScaling
    crosssectionParams['LE_adhesive'] = allAdhesiveThickness * thicknessScaling
    crosssectionParams['TE_adhesive'] = allAdhesiveThickness * thicknessScaling
    #Adhesive width parameters
    crosssectionParams['Web_adhesive_width'] = 0.03 * blade.ichord[iStation]
    crosssectionParams['webAdhesiveOffset'] = 1 * thicknessScaling
    crosssectionParams['webAdhesiveOverhang'] = 1 * thicknessScaling
    crosssectionParams['LE_adhesive_width'] = 0.025 * blade.ichord[iStation] * geometryScaling
    crosssectionParams['TE_adhesive_width'] = 0.025 * blade.ichord[iStation] * geometryScaling
    crosssectionParams['maxWebImperfectionDistance'] = 0.0001 * blade.chord[iStation] * geometryScaling
    
    #Parameters that vary with span but are not part of the study 
    crosssectionParams['minimumTransitionLength']=blade.ichord[iStation]*0.002*geometryScaling;
    
    
    if blade.getprofileTEtype(iStation) == 'flat':
        isFlatback = True
    else:
        isFlatback = False

       
    iStationFirstWeb = np.argwhere(hasWebs)[0][0]
    iStationLastWeb = np.argwhere(hasWebs)[-1][0]
    if hasWebs[iStation] == True:
        webNumber = 1
        aftWebStack = blade.swstacks[webNumber][iStation]
        webNumber = 0
        foreWebStack = blade.swstacks[webNumber][iStation]
    else:
        if iStation < iStationFirstWeb:
            iWebStation=iStationFirstWeb
            
        elif iStationLastWeb == len(blade.ispan) - 1-1:
            iWebStation=iStationLastWeb
        else:
            raise Exception('assuming web ends at last station for now. ')
            
        webNumber = 1
        aftWebStack = blade.swstacks[webNumber][iWebStation]
        webNumber = 0
        foreWebStack = blade.swstacks[webNumber][iWebStation]       
    
    lastRoundStation=getLastRoundStation(blade,TEangleLimit)

    #Only save birdsMouthVerts for the right cross-section
    if iStation == iStationFirstWeb:
        birdsMouthVerts=writeCubitCrossSection(iStation,blade,hasWebs[iStation],aftWebStack,foreWebStack,iLE,crosssectionParams,geometryScaling,thicknessScaling,isFlatback,lastRoundStation,materialIDsUsed)
    else:
        writeCubitCrossSection(iStation,blade,hasWebs[iStation],aftWebStack,foreWebStack,iLE,crosssectionParams,geometryScaling,thicknessScaling,isFlatback,lastRoundStation,materialIDsUsed)
        birdsMouthVerts=[]

    
    cubit.cmd(f'delete curve all with Is_Free except {spanwiseMatOriCurve}')
    #if crossSectionSurfaceCT>maxCrossSectionSurfaceCT:
    #    maxCrossSectionSurfaceCT=crossSectionSurfaceCT
    
    #Chord line for rotation of cross-section for homogenization
    if model2Dor3D.lower() == '2d':
#         #Blocks
        for imat,material in enumerate(blade.materials):
            cubit.cmd(f'block {imat+1} add surface with name "*material{imat}*"')
            
        addColor(blade,'surface')
        
        create_vertex(blade.geometry[0,0,iStation]*geometryScaling,blade.geometry[0,1,iStation]*geometryScaling,blade.geometry[0,2,iStation]*geometryScaling)
        TEvert=get_last_id("vertex")
        create_vertex(blade.geometry[iLE-1,0,iStation]*geometryScaling,blade.geometry[iLE-1,1,iStation]*geometryScaling,blade.geometry[iLE-1,2,iStation]*geometryScaling)
        LEvert=get_last_id("vertex")

        cubit.cmd(f'create curve vertex {TEvert} {LEvert}')
        coords=cubit.vertex(TEvert).coordinates()
        tangent=cubit.curve(get_last_id("curve")).tangent(coords)
        tangentDirection=vectNorm(list(tangent))  #Unit vector of tangent
        crossSectionRotationAngle=math.atan2(tangentDirection[1],tangentDirection[0])*180/pi
        
        #Undo initial twist
        cubit.cmd(f'rotate Surface all angle {blade.degreestwist[iStation]} about Z include_merged ')
        
        #Undo prebend
        if blade.prebend[iStation] != 0: 
            cubit.cmd(f'move surface all y {-1*blade.prebend[iStation]} include_merged')
            
        #Undo sweep
        if blade.sweep[iStation] != 0: 
            raise ValueError('Presweep is untested for cross-sectional meshing' )  
            
        #meshAreas(fid,crosssectionParams.elementShape)
        #writeVABSinputFile(fid,VABSfileNameBase,isFlatback,crosssectionParams.elementShape,crosssectionParams.adhesiveMatID)

        #Mesh the cross-section
        cubit.cmd(f'curve with name "layerThickness*" interval {crosssectionParams["nelPerLayer"]}')
        cubit.cmd('imprint all')
        cubit.cmd('merge all')
        cubit.cmd('set default autosize on')
        cubit.cmd(f'surface with name "*surface*" scheme map')
        cubit.cmd('mesh surface with name "*surface*"')

        if crosssectionParams['elementShape'].lower() == 'tri':
            cubit.cmd(f'surface with name "*surface*" scheme tri')
            
        writeVABSinput()
            


No fatigue exponent found for material: Gelcoat
No fatigue exponent found for material: Adhesive
Warning! material ply thickness medium_density_foam not defined, assuming 1 mm thickness
No fatigue exponent found for material: medium_density_foam
Warning! material ply thickness resin not defined, assuming 1 mm thickness
No fatigue exponent found for material: resin
Warning! material ply thickness steel not defined, assuming 1 mm thickness
No fatigue exponent found for material: steel
...deleting 3894 faces from database...
...deleting 8328 edges from database...
...deleting 4432 nodes from database...
Finished Command: reset

Finished Command: undo off

Geometry accuracy changed from 0.000001 to 0.000001
Finished Command: set geometry accuracy 1e-06

Default autosizing WILL NOT be used when entity sizing has not been specified.
Finished Command: set default autosize off


Creation of Curve 1 (Curve 1) Successful.
Finished Command: create curve spline vertex 1 2 3 4 5 6 7 8 9 10 11 12 13

Created Curve(s): 97
Successfully copied 1 Curve(s).
Finished Command: curve 56 copy

Finished Command: split curve 97 distance 0.0097569007383996 from start

2 VERTS CAPTURED ...
2 VERTS CAPTURED ...

Creation of Curve 100 (Curve 100) Successful.
Finished Command: create curve offset curve 96 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 96 100


Deleted 1 free curves
Finished Command: delete curve 100

Deleted Volume 2
Finished Command: delete body 2


Creation of Curve 105 (Curve 105) Successful.
Finished Command: create curve offset curve 99 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 99 105


Deleted 1 free curves
Finished Command: delete curve 105

Deleted Volume 3
Finished Command: delete body 3

Created Curve(s): 110
Successfully copied 1 Curve(s).
Finished Command: curve 96 copy


Creation of Curve 111 (Curve 111) Successful.
Finished Command: create curve offset curve 110 distance 0.0001

Creati


Creation of Curve 168 (Curve 168) Successful.
Finished Command: create curve offset curve 127 distance -0.021 extended

2 VERTS CAPTURED ...
Created new "combined" curve 170
Finished Command: create curve combine curve 169 168


Deleted 2 free curves
Finished Command: delete curve 168 169

Created new Vertex 570
Finished Command: create vertex atintersection curve 165 170

Finished Command: split curve 170 at vertex 570


Deleted 1 free curves
Finished Command: delete curve 171

Created new Vertex 575
Finished Command: create vertex on curve 134 fraction 0.333333333333333 from start

Created new Vertex 576
Finished Command: create vertex on curve 135 fraction 0.333333333333333 from start

Created new Vertex 577
Finished Command: create vertex on curve 136 fraction 0.333333333333333 from start

Created new Vertex 578
Finished Command: create vertex on curve 137 fraction 0.333333333333333 from start

Created new Vertex 579
Finished Command: create vertex on curve 138 fraction 0.33333333


Creation of Curve 195 (Curve 195) Successful.
Finished Command: create curve offset curve 194 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 194 195


Deleted 1 free curves
Finished Command: delete curve 195

Deleted Volume 8
Finished Command: delete body 8


Creation of Curve 200 (Curve 200) Successful.
Finished Command: create curve offset curve 90 distance -0.004 extended


Creation of Curve 201 (Curve 201) Successful.
Finished Command: create curve offset curve 200 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 200 201


Deleted 1 free curves
Finished Command: delete curve 201

Deleted Volume 9
Finished Command: delete body 9


Creation of Curve 206 (Curve 206) Successful.
Finished Command: create curve offset curve 194 distance -0.021 extended


Creation of Curve 207 (Curve 207) Successful.
Finished Command: create curve offset curve 194 distance -0.057 extended

ERROR: No intersections found between cu


Creation of Curve 266 (Curve 266) Successful.
Finished Command: create curve offset curve 264 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 264 266


Deleted 1 free curves
Finished Command: delete curve 266

Deleted Volume 23
Finished Command: delete body 23


Creation of Curve 271 (Curve 271) Successful.
Finished Command: create curve offset curve 263 distance 0.0001

Creating skin surface...
Finished Command: create surface skin curve 263 271


Deleted 1 free curves
Finished Command: delete curve 271

Deleted Volume 24
Finished Command: delete body 24


Creation of Curve 276 (Curve 276) Successful.
Finished Command: create curve offset curve 264 distance -0.021 extended

2 VERTS CAPTURED ...

Creation of Curve 277 (Curve 277) Successful.
Finished Command: create curve offset curve 263 distance -0.021 extended

2 VERTS CAPTURED ...

Creation of Curve 278 (Curve 278) Successful.
Finished Command: create curve vertex 815 816


Creation of Curve 2

...


Deleted 136 free curves
Finished Command: delete curve all with Is_Free except 1

ERROR: No block with ID 1 was found
ERROR: No entities specified.
Added Surface 167 to block 2
Added Surface 168 to block 2
Added Surface 169 to block 2
Added Surface 170 to block 2
Added Surface 171 to block 2
Added Surface 172 to block 2
Added Surface 174 to block 2
Added Surface 177 to block 2
Added Surface 180 to block 2
Added Surface 183 to block 2
Added Surface 186 to block 2
Added Surface 189 to block 2
Added Surface 192 to block 2
Added Surface 195 to block 2
Added Surface 198 to block 2
Added Surface 201 to block 2
Added Surface 204 to block 2
Added Surface 207 to block 2
Added Surface 210 to block 2
Added Surface 213 to block 2
Added Surface 216 to block 2
Added Surface 219 to block 2
Finished Command: block 2 add surface with name "*material1*"

Added Surface 16 to block 3
Added Surface 30 to block 3
Added Surface 36 to block 3
Added Surface 40 to block 3
Added Surface 44 to block 3
Added Surf

 Moved Body 104:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 105:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 108:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 109:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 110:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 111:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 112:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 113:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 114:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 115:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 116:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 118:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 119:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 120:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Body 122:  x 0.000000e+00  y -1.742505e-01  z 0.000000e+00
 Moved Bod

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh




    (For example, the Shape metric for Face 3198 is 0.0995027 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 3211 is 0.180261 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 3222 is 0.0889888 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 3231 is 0.0746549 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 3240 is 0.163636 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    | 

Finished Command: mesh surface with name "*surface*"



In [32]:
cubit.cmd(f'save as "python0.cub" overwrite')

Progress
Saving CUBIT file '/home/ecamare/myprojects/bar/cubitDev/python/python0.cub'
0   |    |    |    |    50   |    |    |    |  100
*********************************************
Exported: 164 Volumes
            2 Curves

*

True

****
Successfully saved CUBIT file '/home/ecamare/myprojects/bar/cubitDev/python/python0.cub'
Finished Command: save as "python0.cub" overwrite



In [59]:
repr(blade.materials[0])

'<pynumad.objects.Material.Material object at 0x7f6b67df9b20>'

In [6]:
#cubit.cmd('open "/home/ecamare/myprojects/bar/cubitDev/python/python0.cub"')
def getOrderedList(partName):


    orderedList=[]
    surfacesToConnect=[1] #Initialize to enter loop
    iSurface=-1 #Initialize 
    while surfacesToConnect:
        iSurface+=1
        parseString=f'with name "*{partName}*surface{iSurface+1}"'
        surfacesToConnect=parse_cubit_list('surface', parseString)
        
        if surfacesToConnect:
            orderedList.append(surfacesToConnect)

    return orderedList
def makeSpanwiseSplines(orderedList):
    spanwiseSplines=[]
    for alignedSurfaces in orderedList:
        tempList=[]
        for iPoint in range(4):
            vertexList=[]
            for index, iSurface in enumerate(alignedSurfaces):
                vertexID=surfaceDict[iSurface]['verts'][iPoint]
                vertexList.append(vertexID)
                vertexName=cubit.get_entity_name("vertex",vertexID)

            curve=cubit.cmd(f'create curve spline vertex {l2s(vertexList)}')
            tempList.append(get_last_id("curve"))
        spanwiseSplines.append(tempList)
    return spanwiseSplines
    
def makeOneVolume(currentSurfaceID,nextSurfaceID,spanwiseSplinesForAvolume,surfaceDict):
    cubit.cmd(f'surface {currentSurfaceID} copy')
    currentSurfaceIDcopy=get_last_id("surface")
    
    cubit.cmd(f'surface {nextSurfaceID} copy')
    nextSurfaceIDcopy=get_last_id("surface")
    
    currentSurface=cubit.surface(currentSurfaceID)
    nextSurface=cubit.surface(nextSurfaceID)
    
    currentSurfaceCurves=surfaceDict[currentSurfaceID]['curves']
    nextSurfaceCurves=surfaceDict[nextSurfaceID]['curves']
    
    currentSurfaceVerteces=surfaceDict[currentSurfaceID]['verts']
    nextSurfaceVerteces=surfaceDict[nextSurfaceID]['verts'] 
    
    spanwiseSplinesForAvolume.append(spanwiseSplinesForAvolume[0]) #Make list circle back
    
    transverseSurfaceIDs=[]
    for iCurve in range(len(currentSurfaceCurves)):
        cubit.cmd(f'create surface curve {currentSurfaceCurves[iCurve]} {spanwiseSplinesForAvolume[iCurve]} {nextSurfaceCurves[iCurve]} {spanwiseSplinesForAvolume[iCurve+1]}')
        transverseSurfaceIDs.append(get_last_id("surface"))
        
    surfName=cubit.get_entity_name("surface", currentSurface.id()).split('_')
    regex = re.compile('layer')
    layerName = [string for string in surfName if re.match(regex, string)][0]
    stringName=layerName+'_bottomFace'
    cubit.cmd(f'surface {transverseSurfaceIDs[0]} rename "{stringName}"')
    stringName=layerName+'_topFace'
    cubit.cmd(f'surface {transverseSurfaceIDs[2]} rename "{stringName}"')
    
    #cubit.cmd(f'save as "python1.cub" overwrite') 
    #raise Exception(f'Volume "{volumeName}" creation failed')
    #Create Volume
    #nStart=get_last_id("volume")
    cubit.cmd(f'create volume surface {currentSurfaceIDcopy} {nextSurfaceIDcopy} {l2s(transverseSurfaceIDs)} noheal')
    #nEnd=get_last_id("volume")
    #print(f'nStart: {nStart}, nEnd: {nEnd}')

    volumeName=cubit.get_entity_name("surface",currentSurfaceID)
    if len(cubit.volume(get_last_id("volume")).surfaces())<6:
        print(f'\n\n ERROR with:\n\n create volume surface {currentSurfaceIDcopy} {nextSurfaceIDcopy} {l2s(transverseSurfaceIDs)} ')
        print(f'currentSurfaceIDcopy: {currentSurfaceIDcopy}')
        print(f'nextSurfaceIDcopy: {nextSurfaceIDcopy}')
        print(f'spanwiseSplinesForAvolume: {spanwiseSplinesForAvolume}')
        cubit.cmd(f'save as "python1.cub" overwrite') 
        raise Exception(f'Volume "{volumeName}" creation failed')
    
    volumeName=volumeName.replace('surface','volume')
    cubit.cmd(f'volume {get_last_id("volume")} rename "{volumeName}"')
    
def getspanwiseSplinesForAvolume(iSpan,nCrossSections,spanwiseSplinesForOneSurface,nextSurfaceVerteces):
    #Split off spanwise curves for a single volume and store them
    if iSpan<nCrossSections-2:
        spanwiseSplinesForAvolume=[]
        temp=[]
        for iCurve,curveID in enumerate(spanwiseSplinesForOneSurface):
            cubit.cmd(f'split curve {curveID} at vertex {nextSurfaceVerteces[iCurve]}')
            temp.append(get_last_id("curve"))
            spanwiseSplinesForAvolume.append(get_last_id("curve")-1)
        spanwiseSplinesForOneSurface=temp
    else:
        spanwiseSplinesForAvolume=spanwiseSplinesForOneSurface
    return spanwiseSplinesForAvolume,spanwiseSplinesForOneSurface
# def assignIntervals(volID,nIntervals):
#     thicknessCurveID=cubit.volume(volID).curves()[1].id()
#     #cubit.cmd(f'locate curve {thicknessCurveID} ')
#     cubit.cmd(f'curve {thicknessCurveID} interval {nIntervals}')
    
def makeAeroshell(orderedList,meshVolList):
    #nIntervals=3
    spanwiseSplines=makeSpanwiseSplines(orderedList)
    nCrossSections=len(orderedList[0])
    nPartSurfaceIDs=len(orderedList)
    if nCrossSections>1:
        for iSpan in range(nCrossSections-1):
            for partSurfaceIDs in range(nPartSurfaceIDs):
                currentSurfaceID=orderedList[partSurfaceIDs][iSpan]
                nextSurfaceID=orderedList[partSurfaceIDs][iSpan+1]
                spanwiseSplinesForAvolume,spanwiseSplines[partSurfaceIDs]=getspanwiseSplinesForAvolume(iSpan,nCrossSections,spanwiseSplines[partSurfaceIDs],surfaceDict[nextSurfaceID]['verts'])
                makeOneVolume(currentSurfaceID,nextSurfaceID,spanwiseSplinesForAvolume,surfaceDict)
                meshVolList.append(get_last_id("volume"))
                #assignIntervals(get_last_id("volume"),nIntervals)


    return meshVolList


if model2Dor3D.lower() == '3d' and iStationEnd > iStationStart:


    ### ### ### ###
    ### Make volumes along the span. 
    ### ### ### ###
    meshVolList=[]
    
    partName='shell'
    orderedList=getOrderedList(partName)
    if len(orderedList)>0:
        meshVolList=makeAeroshell(orderedList,meshVolList)

    
    partName='web'
    orderedList=getOrderedList(partName)
    if len(orderedList)>0:
        meshVolList=makeAeroshell(orderedList,meshVolList)
    
    partName='roundTEadhesive'
    orderedList=getOrderedList(partName)
    if len(orderedList)>0:
        meshVolList=makeAeroshell(orderedList,meshVolList)
    
    partName='flatTEadhesive'
    orderedList=getOrderedList(partName)
    if len(orderedList)>0:
        meshVolList=makeAeroshell(orderedList,meshVolList)
    
    if crosssectionParams['amplitudeFraction'] and birdsMouthVerts:
        makeBirdsMouth(birdsMouthVerts, crosssectionParams['amplitudeFraction'])
        

    
    cubit.cmd(f'merge volume {l2s(meshVolList)}')
    cubit.cmd(f'reset volume all')
    
    
    cubit.cmd(f'delete surface with Is_Free')
    cubit.cmd(f'curve with name "layerThickness*" interval {crosssectionParams["nelPerLayer"]}')
    cubit.cmd('set default autosize on') 
    cubit.cmd(f'mesh volume {l2s(meshVolList)}')
    cubit.cmd(f'draw volume {l2s(meshVolList)}')
    
    #Blocks
    #for imat,material in enumerate(blade.materials):
    for materialID in materialIDsUsed:
        cubit.cmd(f'block {materialID+1} add volume with name "*material{materialID}*"')
        cubit.cmd(f'block {materialID+1} name "{blade.materials[materialID].name}"')#materialID+1 b/c can't have 0 for block ID
    
    addColor(blade,'volume')
    
    ### Adding Nodesets
    #Root Nodeset
    parseString=f'with name "*station{iStationStart}*"'
    print(f'parseString{parseString}')
    surfaceIDs=parse_cubit_list('surface', parseString)
    cubit.cmd(f'nodeset 1 add surface {l2s(surfaceIDs)} ')
    cubit.cmd(f'nodeset 1 name "root"')


    for iLoop, iStation in enumerate(range(iStationStart+1,iStationEnd,dStation)):
        parseString=f'with name "*station{iStation}*"'
        print(f'parseString{parseString}')
        surfaceIDs=parse_cubit_list('surface', parseString)
        cubit.cmd(f'nodeset {iLoop+2} add surface {l2s(surfaceIDs)} ')
        cubit.cmd(f'nodeset {iLoop+2} name "station{iStation}"')  
    if not list(range(iStationStart+1,iStationEnd,dStation)):
        iLoop=-1
    #Tip Nodeset
    parseString=f'with name "*station{iStationEnd}*"'
    surfaceIDs=parse_cubit_list('surface', parseString)
    cubit.cmd(f'nodeset {iLoop+3} add surface {l2s(surfaceIDs)} ')
    cubit.cmd(f'nodeset {iLoop+3} name "tip"')
    
    
    #Outer mold-line nodeset
    cubit.cmd('draw surf with name "*layer0_bottomFace*"')
    parseString=f'with name "*layer0_bottomFace*"'
    surfaceIDs=parse_cubit_list('surface', parseString)
    cubit.cmd(f'nodeset {iLoop+4} add surface {l2s(surfaceIDs)} ')
    cubit.cmd(f'nodeset {iLoop+4} name "oml"')
    


New Vertex 10653 was created at the location of
   Vertex 658 to be used for start of new curve.
New Vertex 10654 was created at the location of
   Vertex 8118 to be used for end of new curve.

Creation of Curve 3910 (Curve 3910) Successful.
Finished Command: create curve spline vertex 658 3729 8118

New Vertex 10655 was created at the location of
   Vertex 659 to be used for start of new curve.
New Vertex 10656 was created at the location of
   Vertex 8119 to be used for end of new curve.

Creation of Curve 3911 (Curve 3911) Successful.
Finished Command: create curve spline vertex 659 3730 8119

New Vertex 10657 was created at the location of
   Vertex 655 to be used for start of new curve.
New Vertex 10658 was created at the location of
   Vertex 8115 to be used for end of new curve.

Creation of Curve 3912 (Curve 3912) Successful.
Finished Command: create curve spline vertex 655 3726 8115

New Vertex 10659 was created at the location of
   Vertex 654 to be used for start of new curv

New Vertex 10961 was created at the location of
   Vertex 1065 to be used for start of new curve.
New Vertex 10962 was created at the location of
   Vertex 8525 to be used for end of new curve.

Creation of Curve 4064 (Curve 4064) Successful.
Finished Command: create curve spline vertex 1065 4136 8525

New Vertex 10963 was created at the location of
   Vertex 1064 to be used for start of new curve.
New Vertex 10964 was created at the location of
   Vertex 8524 to be used for end of new curve.

Creation of Curve 4065 (Curve 4065) Successful.
Finished Command: create curve spline vertex 1064 4135 8524

New Vertex 10965 was created at the location of
   Vertex 316 to be used for start of new curve.
New Vertex 10966 was created at the location of
   Vertex 7367 to be used for end of new curve.

Creation of Curve 4066 (Curve 4066) Successful.
Finished Command: create curve spline vertex 316 2978 7367

New Vertex 10967 was created at the location of
   Vertex 317 to be used for start of new 

New Vertex 11269 was created at the location of
   Vertex 1783 to be used for start of new curve.
New Vertex 11270 was created at the location of
   Vertex 9664 to be used for end of new curve.

Creation of Curve 4218 (Curve 4218) Successful.
Finished Command: create curve spline vertex 1783 6093 9664

New Vertex 11271 was created at the location of
   Vertex 1784 to be used for start of new curve.
New Vertex 11272 was created at the location of
   Vertex 9665 to be used for end of new curve.

Creation of Curve 4219 (Curve 4219) Successful.
Finished Command: create curve spline vertex 1784 6094 9665

New Vertex 11273 was created at the location of
   Vertex 1756 to be used for start of new curve.
New Vertex 11274 was created at the location of
   Vertex 9637 to be used for end of new curve.

Creation of Curve 4220 (Curve 4220) Successful.
Finished Command: create curve spline vertex 1756 6066 9637

New Vertex 11275 was created at the location of
   Vertex 1755 to be used for start of n

New Vertex 11573 was created at the location of
   Vertex 2211 to be used for start of new curve.
New Vertex 11574 was created at the location of
   Vertex 10100 to be used for end of new curve.

Creation of Curve 4370 (Curve 4370) Successful.
Finished Command: create curve spline vertex 2211 6529 10100

New Vertex 11575 was created at the location of
   Vertex 2212 to be used for start of new curve.
New Vertex 11576 was created at the location of
   Vertex 10101 to be used for end of new curve.

Creation of Curve 4371 (Curve 4371) Successful.
Finished Command: create curve spline vertex 2212 6530 10101

New Vertex 11577 was created at the location of
   Vertex 2192 to be used for start of new curve.
New Vertex 11578 was created at the location of
   Vertex 10077 to be used for end of new curve.

Creation of Curve 4372 (Curve 4372) Successful.
Finished Command: create curve spline vertex 2192 6506 10077

New Vertex 11579 was created at the location of
   Vertex 2191 to be used for star

Updated volume(s): 680
Destroyed volume(s): 675 to 679
Created new volumes: 680 
Finished Command: create volume surface 675 676 677 678 679 680 noheal

Finished Command: volume 680 rename "HPshellStation4_layer0_material4_volume1"

Finished Command: split curve 3914 at vertex 3731

Finished Command: split curve 3915 at vertex 3732

Finished Command: split curve 3916 at vertex 3728

Finished Command: split curve 3917 at vertex 3727

Created Surface(s): 681
Successfully copied 1 Surface(s).
Finished Command: surface 12 copy

Created Surface(s): 682
Successfully copied 1 Surface(s).
Finished Command: surface 238 copy


Creation of Surface 683 (Surface 683) Successful.
This is sheet body 683 (Body 683)
Finished Command: create surface curve 189 4430 1477 4432


Creation of Surface 684 (Surface 684) Successful.
This is sheet body 684 (Body 684)
Finished Command: create surface curve 223 4432 1511 4434


Creation of Surface 685 (Surface 685) Successful.
This is sheet body 685 (Body 685)
Fin

...

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 151767 is 0.0369364 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 152064 is 0.00923709 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 152122 is 0.0179582 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing..

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 155563 is 0.0370677 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 155612 is 0.0351269 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 155856 is 0.0370637 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 155962 is 0.00926994 .)
    The threshold for a Face is 0.2

Gener

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100


    (For example, the Shape metric for Face 171798 is 0.00899701 .)
    The threshold for a Face is 0.2

Generating Mesh
Processing...
0   |    |    |    |    50   |    |    |    |  100

Volumes 680, 686, 692, 698, 704, 710, 716, 722, 728, 734, 740, 746, 752, 758,
   764, 770, 776, 782, 788, 794, 800, 806, 812, 818, 824, 830, 836, 842, 848,
   854, 860, 866, 872, 878, 884, 890, 896, 902, 908, 914, 920, 926, 932, 938,
   944, 950, 956, 962, 968, 974, 980, 986, 992, 998, 1004, 1010, 1016, 1022,
   1028, 1034, 1040, 1046, 1052, 1058, 1064, 1070, 1076, 1082, 1088, 1094, 1100,
   1106, 1112, 1118, 1124, 1130, 1136, 1142, 1148, 1154, 1160, 1166, 1172, 1178,
   1184, 1190, 1196, 1202, 1208, 1214, 1220, 1226, 1232, 1238, 1244, 1250, 1256,
   1262, 126

parseStringwith name "*station4*"
Added Surface 675 to nodeset 1
Added Surface 681 to nodeset 1
Added Surface 687 to nodeset 1
Added Surface 693 to nodeset 1
Added Surface 699 to nodeset 1
Added Surface 705 to nodeset 1
Added Surface 711 to nodeset 1
Added Surface 717 to nodeset 1
Added Surface 723 to nodeset 1
Added Surface 729 to nodeset 1
Added Surface 735 to nodeset 1
Added Surface 741 to nodeset 1
Added Surface 747 to nodeset 1
Added Surface 753 to nodeset 1
Added Surface 759 to nodeset 1
Added Surface 765 to nodeset 1
Added Surface 771 to nodeset 1
Added Surface 777 to nodeset 1
Added Surface 783 to nodeset 1
Added Surface 789 to nodeset 1
Added Surface 795 to nodeset 1
Added Surface 801 to nodeset 1
Added Surface 807 to nodeset 1
Added Surface 813 to nodeset 1
Added Surface 819 to nodeset 1
Added Surface 825 to nodeset 1
Added Surface 831 to nodeset 1
Added Surface 837 to nodeset 1
Added Surface 843 to nodeset 1
Added Surface 849 to nodeset 1
Added Surface 855 to nodeset 1
Added

Added Surface 677 to nodeset 4
Added Surface 683 to nodeset 4
Added Surface 689 to nodeset 4
Added Surface 695 to nodeset 4
Added Surface 749 to nodeset 4
Added Surface 755 to nodeset 4
Added Surface 761 to nodeset 4
Added Surface 803 to nodeset 4
Added Surface 821 to nodeset 4
Added Surface 839 to nodeset 4
Added Surface 857 to nodeset 4
Added Surface 875 to nodeset 4
Added Surface 893 to nodeset 4
Added Surface 911 to nodeset 4
Added Surface 929 to nodeset 4
Added Surface 935 to nodeset 4
Added Surface 941 to nodeset 4
Added Surface 983 to nodeset 4
Added Surface 989 to nodeset 4
Added Surface 995 to nodeset 4
Added Surface 1037 to nodeset 4
Added Surface 1043 to nodeset 4
Added Surface 1049 to nodeset 4
Added Surface 1055 to nodeset 4
Added Surface 1109 to nodeset 4
Added Surface 1115 to nodeset 4
Added Surface 1121 to nodeset 4
Added Surface 1163 to nodeset 4
Added Surface 1181 to nodeset 4
Added Surface 1199 to nodeset 4
Added Surface 1217 to nodeset 4
Added Surface 1235 to nodese

In [16]:
curveID

1109

Progress
Saving CUBIT file '/home/ecamare/myprojects/bar/cubitDev/python/tip.cub'
0   |    |    |    |    50   |    |    |    |  100
*********************************************
Exported: 162 Volumes
            1 Curve

*

True

****
Successfully saved CUBIT file '/home/ecamare/myprojects/bar/cubitDev/python/tip.cub'
Finished Command: save as "tip.cub" overwrite



In [4]:
#cubit.cmd(f'export mesh "/home/ecamare/cee/workspace/barSolid/bladeTest/fine_fine.g" overwrite')



Using ExodusII API version 8.03 and DB version 8.03 to export "/home/ecamare/cee/workspace/barSolid/bladeTest/fine_fine.g".
Writing Exodus file
0   |    |    |    |    50   |    |    |    |  100
**************************************************


Executive Exodus summary:
  Number of dimensions     = 3
  Number of element blocks = 5
  Number of sidesets       = 0
  Number of nodesets       = 3
  Number of bc sets        = 1
  Number of elements       = 213363
  Number of nodes          = 236816

Finished writing /home/ecamare/cee/workspace/barSolid/bladeTest/fine_fine.g
Finished Command: export mesh "/home/ecamare/cee/workspace/barSolid/bladeTest/fine_fine.g" overwrite



True

In [ ]:
#cubit.cmd('open "/home/ecamare/myprojects/bar/cubitDev/python/python0.cub"')

def getApproximateThicknessDirectionForVolume(volumeID):
    #This function is used when assigning material orientations
    
    #Get thickness direction tangents
    approximateThicknessDirection=[]
    for currentCurve in cubit.volume(volumeID).curves():
        curveName=cubit.get_entity_name("curve", currentCurve.id())
        if 'layerThickness' in curveName:
            coords=currentCurve.position_from_fraction(0.5)
            approximateThicknessDirection.append(currentCurve.tangent(coords))
    approximateThicknessDirection=np.array(approximateThicknessDirection)
    nThicknessCurves,_ =approximateThicknessDirection.shape

    if nThicknessCurves==4:  #All other cases
        return np.mean(approximateThicknessDirection,0)
    elif nThicknessCurves ==8: #LE adhesive case and round TE adhesive
        return 0
    elif nThicknessCurves ==6: #Web overwrap 
        # Take the mean of all curves with name 'layerThickness'
        mean=np.mean(approximateThicknessDirection,0)
       
        errorList=[]
        for i in range(nThicknessCurves):
            diff=approximateThicknessDirection[i]-mean

            errorList.append(sqrt(dotProd(diff,diff)))
        sortIndex=np.argsort(errorList)[:4] #Take the first four. This discards the two directions with the largest deviation from the average

        return np.mean(approximateThicknessDirection[sortIndex,:],0)
    else:
        raise ValueError('The number of thickness curves in volume is unexpected. Cannot assign material orientation' ) 
 
    return

def getMatOriSurface(volumeID,spanwiseMatOriCurve):
    #This function is used when assigning material orientations
    #This gets returns the surface within a volume that will be used to get surface normals. 
    #The sign +-1 is also returned since some of the surfaces are oriented the wrong way
    
    approximateThicknessDirection=getApproximateThicknessDirectionForVolume(volumeID)
       
    #Create a list of surface IDs in the given volume
    surfaceIDs=[]
    volumeSurfaces=cubit.volume(volumeID).surfaces()
    for currentSurface in volumeSurfaces:
        surfaceIDs.append(currentSurface.id())
        
    #Eliminate surfaces that have two curves named thickness:
    surfaceCT=0
    for currentSurface in volumeSurfaces:
        curveCT=0 #Counts the number of curves in the surface with name 'layerThickness'
        for currentCurve in currentSurface.curves():
            curveName=cubit.get_entity_name("curve", currentCurve.id())
            if 'layerThickness' in curveName:
                curveCT+=1

        if curveCT>=2:
            surfaceCT+=1
            surfaceIDs.remove(currentSurface.id())
    

    #surfaceIDs now has the list of surfaces w/o thickness curves
    if len(surfaceIDs)==2 or len(surfaceIDs)==1:
        if len(surfaceIDs)==2:
            surfaceName=cubit.get_entity_name("surface", surfaceIDs[0])
            if 'topFace' in surfaceName:
                surfaceID=surfaceIDs[0]
            else:
                surfaceID=surfaceIDs[-1]
        elif len(surfaceIDs)==1: #Web overwrap 
            surfaceID=surfaceIDs[0]
            
        coords=cubit.get_center_point("surface", surfaceID)
        surfaceNormal=cubit.surface(surfaceID).normal_at(coords)

        if dotProd(surfaceNormal,approximateThicknessDirection) >0:
            sign=1.0
        else:
            sign=-1.0
    elif len(surfaceIDs)==0: #LE adhesive and/or TE adhesive for round cross-sections
        #print(f'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~volumeID {volumeID}')
        surfaceID=False
        sign=1.0

    else:
        raise ValueError('The number of thickness curves in volume is unexpected. Cannot assign material orientation' ) 


    return surfaceID, sign


parseString=f'in volume with name "*volume*"'
allVolumeIDs=parse_cubit_list('volume', parseString)

for iVol, volumeID in enumerate(allVolumeIDs):
    surfIDforMatOri,sign=getMatOriSurface(volumeID,spanwiseMatOriCurve)
    

    for iEl,elementID in enumerate(get_volume_hexes(volumeID)):

        coords=cubit.get_center_point("hex", elementID)

        cubit.create_vertex(coords[0],coords[1],coords[2])
        iVert1=get_last_id("vertex")
        if surfIDforMatOri:  
            surfaceNormal=sign*np.array(get_surface_normal_at_coord(surfIDforMatOri,coords))


            curveLocationForTangent=cubit.curve(spanwiseMatOriCurve).closest_point(coords)
            x=cubit.curve(spanwiseMatOriCurve).tangent(curveLocationForTangent)[0]
            y=cubit.curve(spanwiseMatOriCurve).tangent(curveLocationForTangent)[1]
            z=cubit.curve(spanwiseMatOriCurve).tangent(curveLocationForTangent)[2]
            spanwiseDirection=vectNorm([x,y,z])



            perimeterDirection=crossProd(surfaceNormal,spanwiseDirection)
        else:
            perimeterDirection=[1,0,0]
            surfaceNormal=[0,1,0]


        length=0.1
        cubit.create_vertex(coords[0]+length*perimeterDirection[0],coords[1]+length*perimeterDirection[1],coords[2]+length*perimeterDirection[2])
        iVert2=get_last_id("vertex")
        cubit.cmd(f'create curve vertex {iVert1} {iVert2}')
#cubit.cmd(f'save as "python.cub" overwrite') 

In [ ]:
cubit.cmd('open "/home/ecamare/myprojects/bar/cubitDev/python/python0.cub"')
cubit.cmd('open "/home/ecamare/myprojects/bar/cubitDev/python/python0.cub"')


cubit.cmd(f'save as "python0.cub" overwrite')


cubit.cmd(f'save as "python1.cub" overwrite') 

cubit.cmd(f'save as "python.cub" overwrite') 

In [3]:
blade.materials[0].__dict__

{'name': 'Gelcoat',
 'type': 'isotropic',
 'layerthickness': 0.5,
 'ex': 3440000000.0,
 'ey': 3440000000.0,
 'ez': 3440000000.0,
 'gxy': 1323000000.0,
 'gyz': 1323000000.0,
 'gxz': 1323000000.0,
 'prxy': 0.3,
 'pryz': 0.3,
 'prxz': 0.3,
 'density': 1235.0,
 'drydensity': 1235.0,
 'uts': 10000000000.0,
 'ucs': -10000000000.0,
 'uss': 10000000000.0,
 'xzit': 0.3,
 'xzic': 0.25,
 'yzit': 0.3,
 'yzic': 0.25,
 'g1g2': 0.2411764705882353,
 'alp0': 53.0,
 'etat': 0.2867453857588078,
 'etal': -0.7610479585895453,
 'm': None,
 'gamma_mf': None,
 'gamma_ms': None,
 'reference': []}

In [9]:
templateFileName='sm.i.template'
adagioFileName='sm.i'



materialLines=f''
blockLines=f''
for materialID in materialIDsUsed:
    material=blade.materials[materialID]
    print(material.name)
    materialLines+=f'begin property specification for material {material.name}\n'
    materialLines+=f'   DENSITY      = {material.density}\n'
    materialLines+='    begin parameters for model elastic_orthotropic\n'
    materialLines+='        youngs modulus = 70e9\n'
    materialLines+='        poissons ratio  = 0.33\n'
    materialLines+=f'        E11          = {material.ex}\n'
    materialLines+=f'        E22          = {material.ey}\n'
    materialLines+=f'        E33          = {material.ez}\n'
    materialLines+=f'        NU12         = {material.prxy}\n'
    materialLines+=f'        NU13         = {material.prxz}\n'
    materialLines+=f'        NU23         = {material.pryz}\n'
    materialLines+=f'        G12          = {material.gxy}\n'
    materialLines+=f'        G13          = {material.gxz}\n'
    materialLines+=f'        G23          = {material.gyz}\n'

 
    
    materialLines+='\n' 
    materialLines+='        # Coordinate system\n'
    materialLines+='        COORDINATE SYSTEM = sysR\n' 
    materialLines+='    end parameters for model elastic_orthotropic\n' 
    materialLines+=f'end property specification for material {material.name}\n' 
    materialLines+='\n\n' 
    

    blockLines+=f'begin parameters for block {material.name}\n'
    blockLines+=f'    material {material.name}\n'
    blockLines+=f'    solid mechanics use model elastic_orthotropic\n'
    blockLines+=f'end parameters for block {material.name}\n'
    blockLines+='\n\n' 
    


copy_and_replace(templateFileName, adagioFileName,
    {
        'BLADE_MATERIALS': materialLines,
        'BLADE_BLOCKS': blockLines,
    })



Adhesive
glass_uni
glass_biax
glass_triax
medium_density_foam


In [5]:
templateFileName='sd.i.template'
adagioFileName='sd.i'



materialLines=f''
blockLines=f''
for materialID in materialIDsUsed:
    material=blade.materials[materialID]
    print(material.name)
    materialLines+=f'material {material.name}\n'
    materialLines+=f'    E11          = {material.ex}\n'
    materialLines+=f'    E22          = {material.ey}\n'
    materialLines+=f'    E33          = {material.ez}\n'
    materialLines+=f'    NU12         = {material.prxy}\n'
    materialLines+=f'    NU13         = {material.prxz}\n'
    materialLines+=f'    NU23         = {material.pryz}\n'
    materialLines+=f'    G12          = {material.gxy}\n'
    materialLines+=f'    G13          = {material.gxz}\n'
    materialLines+=f'    G23          = {material.gyz}\n'
    materialLines+='\n' 

    materialLines+=f'end \n' 
    materialLines+='\n\n' 
    

    blockLines+=f'block {material.name}\n'
    blockLines+=f'    material {material.name}\n'
    blockLines+=f'end \n'
    blockLines+='\n\n' 
    


copy_and_replace(templateFileName, adagioFileName,
    {
        'BLADE_MATERIALS': materialLines,
        'BLADE_BLOCKS': blockLines,
    })



Adhesive
glass_uni
glass_triax


In [ ]:

cubit.cmd(f'save as "python.cub" overwrite') 


In [ ]:
volumeID
getMatOriSurface(volumeID)
centroid = cubit.get_center_point("hex", 234)
centroid

In [ ]:
cubit.cmd(f'save as "python.cub" overwrite') 


In [ ]:
blade=pynumad.Blade()
blade.read_yaml('myBlade.yaml')


In [ ]:
for i in [1, 2, 3, 4, 5, 6]:
    print(f'{blade.materials[i].name} {blade.materials[i].ex/1e10}')
    